# Modelo Extracao Unificada V2

### Import

In [1]:
import os
import csv
import json
import shutil
from io import StringIO
from pathlib import Path
from PIL import Image
from pdf2image import convert_from_path
import matplotlib.pyplot as plt

import cv2
from PIL import Image
import pytesseract

import fitz  # Módulo PyMuPDF
import re
from fuzzywuzzy import fuzz
from unidecode import unidecode
from unicodedata import normalize
import PyPDF2

import pandas as pd

from PIL import ImageFont
from PIL import Image, ImageDraw

from datetime import datetime, timezone, timedelta

from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar

import modules.ExtracaoPdf as Extc
import modules.cronometro as cron

### Config

In [2]:
### PRESTAR ATENCAO
modelo = 'mage_1'


#### IMPORTANTE - Nro Batch
batch_name = "Batch_9"


# 1. path para documentos PDF (omelhor se estiverem dentro de um unico diretorio)
root_pdf_path = "pipeline_extracao_documentos/2_documentos_para_extracao"

# 2. path para documentos PDF que podem estar aguardando para serem processados
root_pdf_aguardando_path = "pipeline_extracao_documentos/3_tratamento_excecoes/pdf_aguardando_processar"

documentos_extracao_path = "pipeline_extracao_documentos/2_documentos_para_extracao"

# 3. path para documentos PDF externos para serem processados
root_external_pdf_path = "content_from_pdftool/data/data_pdf/NF_para_processamento/NFRJ_PDF_para _ocr"
# 4. path para documentos PDF PESQUISAVEIS externos para serem processados
root_external_pdf_pesquisavel_path = "content_from_pdftool/data/data_pdf/NF_processadas/NFRJ/fwdnotasfiscaisemitidaslmpadalegal"

# 5. path para imagem padrao
image_resized_path = 'pipeline_extracao_documentos/6_geral_administacao/images/processadas'

# 6. path para log
log_path = 'pipeline_extracao_documentos/6_geral_administacao/logs'

# 7. path para arquivos json
json_path = "pipeline_extracao_documentos/5_documentos_processados/jsons"

# 8. path para NFs processadas
nf_processada_path = "pipeline_extracao_documentos/5_documentos_processados"

#### paths de objetos para criacao/gestao (dicionarios/datasets)
# 9. path para modelos
nf_model_path = "pipeline_extracao_documentos/6_geral_administacao/modelos/frames_nf_v7.xlsx"

# 10. path para dicionario de modelos
model_dict_path = "pipeline_extracao_documentos/6_geral_administacao/modelos/models.csv"

# 11. path para datasets CNAE e Itens de Serviço
nf_datasets_path = "pipeline_extracao_documentos/6_geral_administacao/datasets"



# VERIFICAR
tgt_imagens = "pipeline_extracao_documentos/6_geral_administacao/images"

# 12. poppler path
poppler_path = "/home/dani-boy/miniconda3/envs/tables-detr/bin"

# 13. path para config Tesseract
tessdata_dir_config = '--tessdata-dir "/home/dani-boy/miniconda3/envs/tables-detr/share/tessdata/" --user-patterns "novo_modelo/modelos/user-patterns2.txt" --dpi 600 --oem 3 --psm 6'


## Tratamento template do modelo

In [3]:
#Le a planilha e cria do DF
frames_nf_v4_df = pd.read_excel(nf_model_path)

In [4]:
frames_nf_v4_df

,model,seq,type,color,box,t_value,father,label,section_json,reference,x0,y0,x1,y1,Largura,Altura,%
0,mage_1,1,document,NaN,NaN,NaN,NaN,modelo_prefeitura_mage,NaN,NaN,0.0,0.0,2067.0,2923.0,2067.0,2923.0,NaN
1,mage_1,2,boundaries,green,yes,NaN,modelo_prefeitura_mage,boundaries_modelo_prefeitura_mage,NaN,NaN,144.0,99.0,1925.0,2666.0,1781.0,2567.0,100.000000
2,mage_1,3,section,red,yes,NaN,boundaries_modelo_prefeitura_mage,1_section_cabecalho_nf,1 - CABECALHO,NaN,0.0,0.0,2067.0,380.0,2067.0,380.0,14.803272
3,mage_1,4,frame,purple,yes,NaN,1_section_cabecalho_nf,1_frame_prefeitura_nf,1 - CABECALHO,NaN,406.0,0.0,1540.0,380.0,1030.0,380.0,NaN
4,mage_1,5,sframe_field,NaN,no,string,1_frame_prefeitura_nf,nome_prefeitura,1 - CABECALHO,PREFEITURA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,mage_1,73,field_box,orange,yes,string,5_frame_inf_criticas,issqn_retido,9. OUTRAS INFORMAÇOES / CRITICAS,ISSQN RETIDO,1066.0,2425.0,1328.0,2521.0,262.0,96.0,NaN
73,mage_1,74,field_box,orange,yes,string,5_frame_inf_criticas,local_pretacao_servico,9. OUTRAS INFORMAÇOES / CRITICAS,LOCAL. PRESTAÇÃO SERVIÇO,1328.0,2425.0,1638.0,2521.0,310.0,96.0,NaN
74,mage_1,75,field_box,orange,yes,string,5_frame_inf_criticas,local_incidencia,9. OUTRAS INFORMAÇOES / CRITICAS,LOCAL INCIDÊNCIA,1638.0,2425.0,1922.0,2521.0,284.0,96.0,NaN
75,mage_1,76,frame,purple,NaN,NaN,6_section_inf_complementares_criticas,5_frame_observacao,10. OBSERVACOES,uma observação,148.0,2521.0,1922.0,2676.0,1774.0,155.0,NaN


In [5]:
# Cria dicionários para armazenar diferentes tipos de elementos do modelo
document_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'document'].iloc[0]
boundaries_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'boundaries']
sections_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'section']
frames_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'frame']
sframe_fields_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'sframe_field']
field_boxes_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'field_box']

In [ ]:
frames_info

In [ ]:
sframe_fields_info

### Outras funcoes possiveis com frames

In [6]:
# Filtrar o DataFrame para incluir apenas linhas onde a coluna "model" oriundo de: modelo
filtered_frames_info = frames_info[frames_info['model'] == modelo]

for index_frame, row_frame in filtered_frames_info.iterrows():
    frame_model = row_frame['model']
    frame_name = row_frame['label']
    if frame_model == model:
        frame_father = row_frame['label']
        # ... Select specific frames ...
        if frame_father == "2_frame_cnpj_prestador":
        
            
            # Extrai coordenadas para recorte
            x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
            extracted_text_frame = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
            nf_data_prestador = extract_fields_prestador_cnpj(extracted_text_frame)
            
        elif frame_father == "2_frame_inscricao_prestador":
            # Extrai coordenadas para recorte
            x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
            extracted_text_frame = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
            # 1. Prrestador de Servico - INSCRICAO ESTADUAL/MUNICIPAL
            nf_data_prestador_incricao = {}

            # Dividir o texto em linhas
            linhas = extracted_text_frame.split('\n')

            # Inicializar variáveis para armazenar os valores
            inscricao_municipal = None
            inscricao_estadual = None


            # Initialize variables
            inscricao_municipal = ""
            inscricao_estadual = ""

NameError: name 'model' is not defined

In [ ]:
sections_info

In [ ]:
sframe_fields_info

In [ ]:
field_boxes_info

## Funcoes de imagem e extracao

In [7]:
### PRIMEIRAS FUNCOES

# 1. Interacao para pesquisar prefeitura
def pesquisa_texto(texto):
    nome_prefeitura_match = re.search(r'PREFEITURA (.+)', texto)
    if nome_prefeitura_match:
        is_prefeitura = "PREFEITURA " + nome_prefeitura_match.group(1)
        
        return  is_prefeitura
    else:
        raise ValueError("Nao consegui pesquisar")

# 2. Efetua OCR no documento (area parao do texto da NF)
def ocr_RasterPDF_free(image_name, vx0, vy0, vx1, vy1):
    
    analise_pesquisa_nf = {}
    # 1. Definindo as coordenadas do frame
    x0 = vx0
    y0 = vy0
    x1= vx1
    y1 = vy1

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')
    return values, extracted_text_frame 

# 3. Efetua OCR no documento (area parao do texto da NF)
def ocr_RasterPDF(image_name):
    
    analise_pesquisa_nf = {}
    # 1. Definindo as coordenadas do frame
    x0 = 406
    y0 = 0
    x1= 1540
    y1 = 380

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')
    return values, extracted_text_frame 


# Funcao importante - process_line
def process_line(value, reference, label):
    name_match = re.search(fr'{reference} (.+)', value)
    if name_match:
        extracted_value = reference + " " + name_match.group(1)
        return {label: extracted_value}
    return None



# Funcao de conversao e resize do documento
def convertResize_analise(nome_documento, document_path, image_resized_path):
    
    """# 1. remocao do sufixo .pdf
    if doc2convert.split(".")[1].islower():
        nameImage= doc2convert.removesuffix(".pdf")
    else:
        nameImage= doc2convert.removesuffix(".PDF")"""
    
    # 2. construo um novo nome para o documento imagem
    image_resized_name = os.path.join(f'{image_resized_path}/{str(nome_documento)}.jpg')
    
    # 3. Conversao para imagem
    pages = convert_from_path(document_path, 500, poppler_path=poppler_path)
    
    # 4. Verifica se ha mais que uma pagina
    if len(pages) > 1:
        raise ValueError("Erro, documento com mais de uma página")
    else:
        # 5. Iterar pelas páginas e redimensionar
        resized_pages = []
        for page in pages:
            resized_page = page.resize((2067, 2923))
            resized_pages.append(resized_page)
            
        resized_pages[0].save(image_resized_name, 'JPEG')
        
    return resized_pages[0], image_resized_name


# Funcao de pesquisa de modelo
def pequisaModel(image_name):
    
    analise_pesquisa_nf = {}
    # 1. Definindo as coordenadas do frame
    x0 = 406
    y0 = 0
    x1= 1540
    y1 = 380

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')

    # 5. Interacao para pesquisar prefeitura
    for value in values:
        nome_prefeitura_match = re.search(r'PREFEITURA (.+)', value)
        if nome_prefeitura_match:
            nome_prefeitura = "PREFEITURA " + nome_prefeitura_match.group(1)
            return  nome_prefeitura
        else:
            raise ValueError("Nao acho nome de prefeitura")
        
# 1. Funcao de conversao e resize do documento
def convertResize(doc2convert, document_path, image_resized_path):
    
    """# 1. remocao do sufixo .pdf
    if doc2convert.split(".")[1].islower():
        nameImage= doc2convert.removesuffix(".pdf")
    else:
        nameImage= doc2convert.removesuffix(".PDF")"""
    
    # 2. construo um novo nome para o documento imagem
    image_resized_name = os.path.join(f'{image_resized_path}/{str(doc2convert)}.jpg')
    
    # 3. Conversao para imagem
    pages = convert_from_path(document_path, 500, poppler_path=poppler_path)
    
    # 4. Verifica se ha mais que uma pagina
    if len(pages) > 1:
        raise ValueError("Erro, documento com mais de uma página")
    else:
        # 5. Iterar pelas páginas e redimensionar
        resized_pages = []
        for page in pages:
            resized_page = page.resize((2067, 2923))
            resized_pages.append(resized_page)
            
        resized_pages[0].save(image_resized_name, 'JPEG')
        
    return resized_pages[0], image_resized_name



# 2. Pesquisa prefeitura no documento
def pequisaModel(image_name):

    # 1. Definindo as coordenadas do frame
    x0 = 406
    y0 = 0
    x1= 1540
    y1 = 380

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')

    # 5. Interacao para pesquisar prefeitura
    for value in values:
        nome_prefeitura_match = re.search(r'PREFEITURA (.+)', value)
        if nome_prefeitura_match:
            nome_prefeitura = "PREFEITURA " + nome_prefeitura_match.group(1) 
            return  nome_prefeitura        
        


# 3. Ajusta o filename tirando caracteres especiais 
def conv_filename(title):
    
    # Divida o título em nome e extensão
    name, extension = title.rsplit('.', 1) if '.' in title else (title, "")

    # Remova acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substitua espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remova quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converter para minúsculas
    filename = filename.lower()

    # Adicione a extensão de volta, se houver
    if extension:
        filename += '.' + extension.lower()

    return filename

# 4. Ajusta o filename tirando caracteres especiais e a
def conv_filename_no_ext(title):
    # Divida o título em nome e extensão (mas ignore a extensão)
    name = title.rsplit('.', 1)[0] if '.' in title else title

    # Remova acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substitua espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remova quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converter para minúsculas
    filename = filename.lower()

    return filename  

# 2. Efetua OCR no documento (area parao do texto da NF)
def ocr_RasterPDF_free(image_name, vx0, vy0, vx1, vy1):
    
    analise_pesquisa_nf = {}
    # 1. Definindo as coordenadas do frame
    x0 = vx0
    y0 = vy0
    x1= vx1
    y1 = vy1

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')
    return values, extracted_text_frame 


# 5. Verifica se PDF e pesquisavel ou nao e grava metadados dele
def is_pdf_searchable_analise(pdf_path):
    try:
        pdf_document = fitz.open(pdf_path)
        pages = pdf_document.page_count
        is_searchable = all(page.get_text("text") != "" for page in pdf_document)
        dados_pdf = pdf_document.metadata
        pdf_document.close()
        return is_searchable, dados_pdf, pages
    except Exception as e:
        print(f"Erro ao verificar o PDF: {e}")
        return False

# 2. Efetua OCR no documento (area parao do texto da NF)
def ocr_RasterPDF_free(image_name, vx0, vy0, vx1, vy1):
    
    analise_pesquisa_nf = {}
    # 1. Definindo as coordenadas do frame
    x0 = vx0
    y0 = vy0
    x1= vx1
    y1 = vy1

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')
    return values, extracted_text_frame 

# Funcao importante - process_line

def process_line(value, reference, label):
    name_match = re.search(fr'{reference} (.+)', value)
    if name_match:
        extracted_value = reference + " " + name_match.group(1)
        return {label: extracted_value}
    return None


def convertResizeAnalise_1page(doc2convert, document_path, image_resized_path):
    
    """# 1. remocao do sufixo .pdf
    if doc2convert.split(".")[1].islower():
        nameImage= doc2convert.removesuffix(".pdf")
    else:
        nameImage= doc2convert.removesuffix(".PDF")"""
    
    # 2. construo um novo nome para o documento imagem
    image_resized_name = os.path.join(f'{image_resized_path}/{str(doc2convert)}.jpg')
    
    # 3. Conversao para imagem
    pages = convert_from_path(document_path, 500, poppler_path=poppler_path)
    
    resized_pages = []
    for page in pages:
        resized_page = page.resize((2067, 2923))
        resized_pages.append(resized_page)
        resized_pages[0].save(image_resized_name, 'JPEG')
        
    return resized_pages[0], image_resized_name

# 2. Leitura do arquivo CSV e criação do dicionário modelos
def create_model_dictionary(model_dict_path):
    model_dictionary = {}
    with open(model_dict_path, 'r') as csvfile:
        csvreader = csv.DictReader(csvfile)
        for row in csvreader:
            prefeitura_name = row['prefeitura']
            model_name = row['model']

            if prefeitura_name not in model_dictionary:
                model_dictionary[prefeitura_name] = model_name
            
            #model_dictionary[prefeitura_name].append(model_name)
    
    return model_dictionary



def extract_text_from_coordinates(image, coordinates, config):
    x0, y0, x1, y1 = coordinates
    frame_image = image.crop((x0, y0, x1, y1))
    extracted_text = pytesseract.image_to_string(frame_image, lang='por', config=config).strip()
    return extracted_text


# secao: 8 - DADOS COMPLEMENTARES & 10. OBSERVACOES
def extract_dados_from_frame(modelo, frame_father, section):
    
    data_dados_frame = {}
    #frame_label = frame_father
    
    # 1. Filtrando o frames_info para buscar os dados de corte
    filtered_frames_info = frames_info[(frames_info['label'] == frame_father) & (frames_info['model'] == modelo)]

    # 2. Filtrando o sframe_fields_info para buscar os dados dos campos que estao nos frames
    filtered_sframe_fields_info = sframe_fields_info[(sframe_fields_info['father'] == frame_father) & (sframe_fields_info['model'] == modelo)]

    for index_frame, row_frame in filtered_frames_info.iterrows():
        
        x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
        extracted_text_frame = extract_text_from_coordinates(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        
                       
                
    return extracted_text_frame        
        
        
def extract_text_from_frame(image, coordinates, config):
    x0, y0, x1, y1 = coordinates
    frame_image = image.crop((x0, y0, x1, y1))
    extracted_text = pytesseract.image_to_string(frame_image, lang='por', config=config).strip()
    return extracted_text

def format_number(number_str):
    # Check for percentage and handle it
    if '%' in number_str:
        number_str = number_str.replace('%', '')
        return float(number_str)  # You can multiply by 100 here if needed

    # Check if the string contains "R$" or a comma, indicating the original format
    if 'R$' in number_str or ',' in number_str:
        # Original format: Remove 'R$', replace dots with nothing, and replace commas with dots
        number_str = number_str.replace('R$', '').replace('.', '').replace(',', '.')
    else:
        # New format: Extract only the numeric part using regex
        number_str = re.findall(r'[\d\.]+', number_str)[-1]

    return float(number_str)

# Funçao de formatacao de numeros
def format_number2(number_str):
    number_str = number_str.replace('R$', '').replace('.', '').replace(',', '.')
    if '%' in number_str:
        number_str = number_str.replace('%', '')
        return float(number_str)  # multiplica por 100 para fields %
    return float(number_str)


def extract_fields_box(modelo, father_value, section):

    data_box_valores = {}
    data_box_valores['secao'] = section
    filtered_boxes_info = field_boxes_info[(field_boxes_info['father'] == father_value) & (field_boxes_info['model'] == modelo)]
    # Iterate nas informações dos boxes de fields e extraia o texto de cada field
    for index_field, row_field in filtered_boxes_info.iterrows():
        x0, y0, x1, y1 = row_field['x0'], row_field['y0'], row_field['x1'], row_field['y1']
        extracted_text_box = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        #print("{:<5} {:<10} {:<30} {:<20} {:<20} {:<7} {:<7} {:<7} {:<7}".format(row_field['seq'], row_field['model'], row_field['father'], row_field['label'], row_field['reference'], row_field['x0'], row_field['y0'], row_field['x1'], row_field['y1'] ))
        # Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
        value = extracted_text_box.split('\n')[-1]
        # Remova qualquer espaço em branco à esquerda ou à direita
        value = value.strip()
        if row_field['t_value'] == 'number':
            # Formate o valor usando a função format_number
            #print("vou verificar valor")
            value = format_number2(value)
            #print(value)
        # Armazene o texto extraído com o rótulo correspondente
        label = row_field['label']
        data_box_valores[label] = value
        
    return data_box_valores

# move NF processadas ok
def move_raster_pdf(document_path, raster_pdf_path, batch_name, doc2convert):
    # Determine the destination directory
    destination_dir = os.path.join(raster_pdf_path, batch_name)

    # Check if the destination directory exists; if not, create it
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    # Determine the destination path including the filename
    destination_path = os.path.join(destination_dir, os.path.basename(document_path))

    # Move the file from the source path to the destination path
    try:
        shutil.move(document_path, destination_path)
        print(f"Sucesso ao mover: {document_path} para: {destination_path}")
        return True, destination_path, None  # Success, destination path, no error
    except Exception as e:
        error_message = f"Erro ao mover: {document_path} para: {destination_path}: {str(e)}"
        print(error_message)
        return False, None, error_message  # Failure, no destination path, error message
        

In [8]:
def simple_page_split(input_pdf_path):
    
    # Lista para guardar paginas
    document_pages = []
    # Abre o PDF
    pdf = fitz.open(input_pdf_path)

    # Número total de páginas no PDF
    total_pages = len(pdf)

    # Nome base para os arquivos de saída
    base_name = input_pdf_path.split('.')[0]  # Remove a extensão do arquivo

    # Loop para criar um novo PDF para cada página
    for page_num in range(total_pages):
        # Cria um novo objeto PDF
        new_pdf = fitz.open()
        # Adiciona a página atual ao novo PDF
        new_pdf.insert_pdf(pdf, from_page=page_num, to_page=page_num)
        # Nome do novo arquivo PDF
        new_pdf_name = f"{base_name}_page_{page_num + 1}.pdf"
        #splited_doc_path = os.path.join(destiny_path, new_pdf_name)
        # Atualiza lista de paginas
        document_pages.append(new_pdf_name)
        # Salva o novo PDF
        new_pdf.save(new_pdf_name)
        # Fecha o novo PDF
        new_pdf.close()

    # Fecha o PDF original
    pdf.close()
    return document_pages

## Pipeline  <mark>PDF Pesquisavel e Raster PDF</mark> 

### Teste <mark>RASTER_PDF</mark>

### dict dados dos documento

In [9]:
# Lista para armazenar as linhas
rows_list = []

# Função recursiva para adicionar linha
def add_row_recursively(rows_list, 
                        index, 
                        Batch, 
                        diretorio_ori, 
                        arquivo_origem, 
                        arquivo_destino, 
                        data_hora, 
                        tipo_doc_pdf, 
                        qtd_paginas
                        ):
    if index == 0:
        return rows_list
    else:
        new_row = {
                    'index': index,
                    'Batch': Batch,
                    'diretorio_origem': diretorio_ori,
                    'nome_arquivo_origem': arquivo_origem,
                    'nome_arquivo_destino': arquivo_destino,
                    'data_processamento': data_hora,
                    'tipo_pdf': tipo_doc_pdf,
                    'qut_paginas': qtd_paginas
                    }
        rows_list.append(new_row)
        
        return add_row_recursively(rows_list, index-1, Batch, diretorio_ori, arquivo_origem, arquivo_destino, data_hora, tipo_doc_pdf, qtd_paginas)

In [10]:
df_processamento = pd.read_excel("pipeline_extracao_documentos/6_geral_administacao/datasets/df_processamento_7.xlsx")   

### cricao dos dados ambiente

In [11]:
#### IMPORTANTE - NRO BATCH PARA TESTE    0 = PDF_PESQUISAVEL | 1 = RASTER_PDF

i_test = 1

modelo = 'mage_1'

model = 'mage_1' 

tipo_pdf = []
tipo_pdf.append('PDF_PESQUISAVEL')
tipo_pdf.append('RASTER_PDF')
tipo_pdf[i_test]


# Tratamento do Path de ORIGEM DO DOCUMENTOS PARA TESTE QUE SERAO MOVIDOS
list_path_test = []
list_path_test.append("pipeline_extracao_documentos/4_area_testes/pdf_pesquisavel_4_test")
list_path_test.append("pipeline_extracao_documentos/4_area_testes/raster_pdf_4_test")
list_path_test[i_test]

# Frame para teste
i_frame = 0

frames_pesquisa = []
# Filtrar o DataFrame para incluir apenas linhas onde a coluna "model" oriundo de: modelo
filtered_frames_info = frames_info[frames_info['model'] == modelo]
for index_frame, row_frame in filtered_frames_info.iterrows():
    frame_name = row_frame['label']
    frames_pesquisa.append(frame_name)

# Nome Batch
batch_name = "Batch_" + str(tipo_pdf[i_test]) + "_" + str(i_frame)

# Nome do arquivo json
nome_formado_json = batch_name +".json"

# Listagem dos frames de pesquisa
i = 0
for frame in frames_pesquisa:
    print(f'seq ={i:>3} | {frame}')
    i += 1
    
if frames_pesquisa[i_frame]:
    print(f'\n\nDados do teste: batch_name: {batch_name} | frame: {frames_pesquisa[i_frame]} | model: {modelo} | tipo_pdf: {tipo_pdf[i_test]}')
    
    
######### PATHS
#1. path formado para busca de pdfs recursiva
root_doc_analise = os.path.join(documentos_extracao_path, batch_name)

#2. path para documentos teste RASTER PDF (ATRIBIDO DA LISTA)
path_test_pdf = list_path_test[1]

#3. path formado para nome do arquivo json
json_file_path = os.path.join(json_path, nome_formado_json)

#Listando paths utilizados
print(f'\nroot_doc_analise: {root_doc_analise}\npath_test_pdf: {path_test_pdf}\njson_file_path: {json_file_path}')       

seq =  0 | 1_frame_prefeitura_nf
seq =  1 | 1_frame_dados_nf
seq =  2 | 2_frame_cnpj_prestador
seq =  3 | 2_frame_inscricao_prestador
seq =  4 | 2_frame_dados_prestador
seq =  5 | 3_frame_cnpj_tomador
seq =  6 | 3_frame_inscricao_tomador
seq =  7 | 3_frame_dados_tomador
seq =  8 | 4_frame_descricao_totais
seq =  9 | 4_frame_valor_total
seq = 10 | 4_frame_cnae_itens_servico
seq = 11 | 5_frame_valores_impostos
seq = 12 | 5_frame_dados_complementares
seq = 13 | 5_frame_inf_criticas
seq = 14 | 5_frame_observacao


Dados do teste: batch_name: Batch_RASTER_PDF_0 | frame: 1_frame_prefeitura_nf | model: mage_1 | tipo_pdf: RASTER_PDF

root_doc_analise: pipeline_extracao_documentos/2_documentos_para_extracao/Batch_RASTER_PDF_0
path_test_pdf: pipeline_extracao_documentos/4_area_testes/raster_pdf_4_test
json_file_path: pipeline_extracao_documentos/5_documentos_processados/jsons/Batch_RASTER_PDF_0.json


### funçao de mover documento de uma area para outra

In [ ]:
# Diretório onde você quer salvar os arquivos extraídos
output_dir = os.path.join(documentos_extracao_path, batch_name)
i_max = 2
for root, dirs, files in os.walk(path_test_pdf):
    #dir_name = os.path.basename(root)
    i = 1
    print(f'1. | {root}  | {dirs} \n')
    for file in files:
        file_path = os.path.join(root, file)
        doc2test = file
        pesquisavel, metadados, paginas = is_pdf_searchable_analise(file_path)
        print(f'\n      iter: {i} diretorio: {dir_name} | Doc: {file} | reno: {doc2test}\n\n      pesquisavel: {pesquisavel}\n      paginas = {paginas}\n')
        
        doc2test_path = os.path.join(root, doc2test)
        destination_path = os.path.join(output_dir, doc2test)
        #destination_path = os.path.join(destination_dir, os.path.basename(document_path))
        #print(f'\n      iter: {i} Documento: {file} | renomeado para: {doc2test}\n\n      mover de: output_dir = {doc2test_path}\n      para: destination_path =  {destination_path}\n')
        if not os.path.exists(output_dir):
           os.makedirs(output_dir) # estou criando o diretorio caso nao exista
           
        # Move the file from the source path to the destination path
        try:
            shutil.move(doc2test_path, destination_path)
            print(f"        Sucesso ao mover: {doc2test_path} para: {destination_path}")
        except Exception as e:
            error_message = f"      Erro ao mover: {doc2test_path} para: {destination_path}: {str(e)}"
            print(error_message)   
        
        if i == i_max: #Define quantidade de tratamento de documentos raster PDF
            break
        i += 1    
    
    
print(f'\ntotal de {i} documentos movidos')        

### ajusta o nome dos documentos e grava em dicionario

In [ ]:
# Ajustando o filename e criando o dicionario
folder_file_dict = {}
rows_list = []
output_dir = os.path.join(documentos_extracao_path, batch_name)
i = 0
for root, dirs, files in os.walk(root_doc_analise):
    folder_name = os.path.basename(root)
    for file in files:
        file_path = os.path.join(root, file)
        pesquisavel, metadados, paginas = is_pdf_searchable_analise(file_path)

        new_name = conv_filename(file)
        new_path_name = os.path.join(root, new_name)
        #print(f'\nfile: {file} | new_name: {new_name} ')
        shutil.move(file_path, new_path_name)
        folder_file_dict.setdefault(folder_name, []).append(new_name)
        time_now = cron.timenow_pt_BR()
        new_row = {
                    'index': i,
                    'Batch': batch_name,
                    'diretorio_origem': folder_name,
                    'nome_arquivo_origem': file,
                    'nome_arquivo_destino': new_name,
                    'data_processamento': time_now,
                    'tipo_pdf': pesquisavel,
                    'qut_paginas': paginas
                    }
        rows_list.append(new_row)
    i += 1

In [ ]:
folder_file_dict

In [ ]:
rows_list

In [ ]:
df_extracao_files = pd.DataFrame(rows_list)
df_extracao_files

### Modelo atual de call de funcoes Raster PDF

In [ ]:
# Frame para teste
i_frame = 0

frames_pesquisa = []
# Filtrar o DataFrame para incluir apenas linhas onde a coluna "model" oriundo de: modelo
filtered_frames_info = frames_info[frames_info['model'] == modelo]
for index_frame, row_frame in filtered_frames_info.iterrows():
    frame_name = row_frame['label']
    frames_pesquisa.append(frame_name)

In [ ]:
data_sframe_info = {}

In [ ]:
sframe_fields_info

In [ ]:
filtered_sframe_fields_info = sframe_fields_info[sframe_fields_info['model'] == modelo]
for index_frame, row_frame in filtered_frames_info.iterrows():
    frame_name = row_frame['label']
    frames_pesquisa.append(frame_name)

In [ ]:
sframe_fields_info

In [ ]:
filtered_sframe_fields_info = sframe_fields_info[(sframe_fields_info['father'] == f_father) & (sframe_fields_info['model'] == model)]

In [ ]:
frames_nf_v4_df

In [ ]:
print(f'\nf_father: {f_father} |  model: {model}')

In [ ]:
sframe_fields_info

In [ ]:
data_sframe_info

In [ ]:

    
    data_dados_frame = {}
    #frame_label = frame_father
    
    # 1. Filtrando o frames_info para buscar os dados de corte
    filtered_sframe_fields_info = sframe_fields_info[(sframe_fields_info['label'] == frame_father) & (sframe_fields_info['model'] == modelo)]

    # 2. Filtrando o sframe_fields_info para buscar os dados dos campos que estao nos frames
    filtered_sframe_fields_info = sframe_fields_info[(sframe_fields_info['father'] == frame_father) & (sframe_fields_info['model'] == modelo)]

    for index_frame, row_frame in filtered_frames_info.iterrows():
        
        x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
        extracted_text_frame = extract_text_from_coordinates(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        
        
        
        label = row_frame['label']
        data_dados_frame[label] = value
        # data_dados_frame['seq'] = row_frame['seq']
        # data_dados_frame['label'] = row_frame['label']
        # data_dados_frame['reference'] = row_frame['reference']
        
                
 

In [ ]:
filtered_frames_info = frames_info[frames_info['model'] == modelo]

In [ ]:
# Filtrar o DataFrame para incluir apenas linhas onde a coluna "model" oriundo de: modelo
filtered_frames_info = frames_info[frames_info['model'] == model]

In [ ]:
section = "1 - CABECALHO"

In [ ]:
filtered_frames_info = frames_info[(frames_info['section_json'] == section) & (frames_info['model'] == model)]

In [ ]:
     sframe_father = srow_frame['father']
        
        sframe_seq = srow_frame['seq']
        sframe_label = srow_frame['label']
        sframe_label = srow_frame['label']

In [ ]:
section = []
# Modelo 1: Sequencia de todos os campos
filtered_frames_info = frames_info[frames_info['model'] == model]
for index_frame, row_frame in filtered_frames_info.iterrows():
    frame_seq = row_frame['seq']
    frame_model = row_frame['model']
    frame_label = row_frame['label']
    frame_section = row_frame['section_json']
    section.append(frame_section)

In [ ]:
section

In [ ]:
# Modelo 1: Sequencia de todos os campos
filtered_frames_info = frames_info[frames_info['model'] == model]
for index_frame, row_frame in filtered_frames_info.iterrows():
    frame_seq = row_frame['seq']
    frame_model = row_frame['model']
    frame_label = row_frame['label']
    frame_section = row_frame['section_json']
    #print(f'\fseq: {frame_seq:>3} | model: {frame_model:>8} | label: {frame_name:>20} | section: {frame_section:>35} ')
    filtered_sframe_fields_info = sframe_fields_info[(sframe_fields_info['model'] == frame_model) & (sframe_fields_info['section_json'] == frame_section) & (sframe_fields_info['father'] == frame_label)]
    for sindex_frame, srow_frame in filtered_sframe_fields_info.iterrows():
        sframe_section = srow_frame['section_json']
        if sframe_section == frame_section:
            sframe_seq = srow_frame['seq']
            sframe_label = srow_frame['label']
            sframe_model = srow_frame['model']
            sframe_reference = srow_frame['reference']
            print(f'\fseq: {sframe_seq:>3} | model: {sframe_model:>8} | label: {sframe_label:>20} | section: {sframe_section:>35}  reference: {sframe_reference:>35} ')

In [ ]:
# MOdelo 1: Sequencia de todos os campos por secao (filtro)

section = "1 - CABECALHO"
filtered_frames_info = frames_info[(frames_info['section_json'] == section) & (frames_info['model'] == model)]
for index_frame, row_frame in filtered_frames_info.iterrows():
    frame_seq = row_frame['seq']
    frame_model = row_frame['model']
    frame_label = row_frame['label']
    frame_section = row_frame['section_json']
    #print(f'\fseq: {frame_seq:>3} | model: {frame_model:>8} | label: {frame_name:>20} | section: {frame_section:>35} ')
    filtered_sframe_fields_info = sframe_fields_info[(sframe_fields_info['model'] == frame_model) & (sframe_fields_info['section_json'] == frame_section) & (sframe_fields_info['father'] == frame_label)]
    for sindex_frame, srow_frame in filtered_sframe_fields_info.iterrows():
        sframe_section = srow_frame['section_json']
        if sframe_section == frame_section:
            sframe_seq = srow_frame['seq']
            sframe_label = srow_frame['label']
            sframe_model = srow_frame['model']
            sframe_reference = srow_frame['reference']
            print(f'\fseq: {sframe_seq:>3} | model: {sframe_model:>8} | label: {sframe_label:>20} | section: {sframe_section:>35}  reference: {sframe_reference:>35} ')
        
   
        
    

In [ ]:
    filtered_boxes_info = field_boxes_info[(field_boxes_info['father'] == father_value) & (field_boxes_info['model'] == modelo)]
    # Iterate nas informações dos boxes de fields e extraia o texto de cada field
    for index_field, row_field in filtered_boxes_info.iterrows():
        x0, y0, x1, y1 = row_field['x0'], row_field['y0'], row_field['x1'], row_field['y1']
        extracted_text_box = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        #print("{:<5} {:<10} {:<30} {:<20} {:<20} {:<7} {:<7} {:<7} {:<7}".format(row_field['seq'], row_field['model'], row_field['father'], row_field['label'], row_field['reference'], row_field['x0'], row_field['y0'], row_field['x1'], row_field['y1'] ))
        # Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
        value = extracted_text_box.split('\n')[-1]
        # Remova qualquer espaço em branco à esquerda ou à direita
        value = value.strip()
        if row_field['t_value'] == 'number':
            # Formate o valor usando a função format_number
            #print("vou verificar valor")
            value = format_number2(value)
            #print(value)
        # Armazene o texto extraído com o rótulo correspondente
        label = row_field['label']
        data_box_valores[label] = value
        
    return data_box_valores

In [ ]:
filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['section_json'] == section) & (frames_nf_v4_df['model'] == model)]

In [ ]:
filtered_frames_nf_v4_df = frames_nf_v4_df[frames_nf_v4_df['model'] == model]

In [ ]:
for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
    frame_seq = row_frame['seq']
    frame_model = row_frame['model']
    frame_label = row_frame['label']
    frame_type = row_frame['type']
    frame_section = row_frame['section_json']
    frame_reference = row_frame['reference']
    print(f'\fseq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | label: {frame_name:>20} | section: {frame_section:>35} {frame_reference:>30}')

In [ ]:

    if frame_model == model:
        frame_father = row_frame['label']
        # ... Select specific frames ...
        if frame_father == "2_frame_cnpj_prestador":

In [ ]:
    # 1. Filtrando o frames_info para buscar os dados de corte
    filtered_frames_info = frames_info[(frames_info['label'] == frame_father) & (frames_info['model'] == modelo)]

    # 2. Filtrando o sframe_fields_info para buscar os dados dos campos que estao nos frames
    filtered_sframe_fields_info = sframe_fields_info[(sframe_fields_info['father'] == frame_father) & (sframe_fields_info['model'] == modelo)]

    for index_frame, row_frame in filtered_frames_info.iterrows():

In [ ]:
frames_pesquisa = []
# Filtrar o DataFrame para incluir apenas linhas onde a coluna "model" oriundo de: modelo
filtered_sframe_fields_info = sframe_fields_info[sframe_fields_info['model'] == modelo]
for index_frame, row_frame in filtered_frames_info.iterrows():
    
    filtered_sframe_fields_info = sframe_fields_info[(sframe_fields_info['father'] == f_father) & (sframe_fields_info['model'] == model)]
    
    
    
    frame_section_name = row_frame['section_json']
    filtered_sframe_fields_info = sframe_fields_info[(sframe_fields_info['secao'] == frame_father) & (sframe_fields_info['model'] == modelo)]
    frame_section_name = row_frame['section_json']
    frame_label_name = row_frame['label']
    print(frame_label_name, frame_section_name)
   

In [ ]:
 frame_label_name = row_frame['label']
    frames_pesquisa.append(frame_name)

In [ ]:
frame2search = frames_pesquisa[i_frame]

In [ ]:
frame2search

In [ ]:
if frames_pesquisa[i_frame]:
    print(f'\n\nDados do teste: batch_name: {batch_name} | frame: {frames_pesquisa[i_frame]} | model: {modelo} | tipo_pdf: {tipo_pdf[i_test]} | path: {root_doc_analise}')

In [ ]:
frames_pesquisa[i_frame]

In [34]:
i_frame = 0
frames_pesquisa = []
filtered_frames_nf_v4_df = frames_nf_v4_df[frames_nf_v4_df['model'] == model]
for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
    frame_seq = row_frame['seq']
    frame_model = row_frame['model']
    frame_label = row_frame['label']
    frame_type = row_frame['type']
    frame_section = row_frame['section_json']
    frame_reference = row_frame['reference']
    frame_father = row_frame['father']
    #if frame_section == "1 - CABECALHO" and frame_type == "frame":
    if frame_section == "1 - CABECALHO":
        frame_father = row_frame['father']
        data_cabecalho = {}
        data_cabecalho['secao'] = frame_section
        if frame_father == "1_frame_prefeitura_nf" and frame_type == "sframe_field":
        #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            print(f'\fseq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>20} | section: {frame_section:>35} {frame_reference:>30}')
            result = {}
            
        elif frame_father == "1_frame_dados_nf" and frame_type == "sframe_field":
            frame_father = row_frame['father']
            print(f'\fseq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>20} | section: {frame_section:>35} {frame_reference:>30}')
            result = {} 
    elif frame_section == "2. PRESTADOR DE SERVIÇO":
        frame_father = row_frame['father']
        data_cabecalho = {}
        data_cabecalho['secao'] = frame_section
        if frame_father == "2_frame_cnpj_prestador" and frame_type == "sframe_field":
        #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            print(f'\fseq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>20} | section: {frame_section:>35} {frame_reference:>30}')
            result = {}
            
        elif frame_father == "2_frame_inscricao_prestador" and frame_type == "sframe_field":
            frame_father = row_frame['father']
            print(f'\fseq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>20} | section: {frame_section:>35} {frame_reference:>30}')
            result = {} 
        elif frame_father == "2_frame_dados_prestador" and frame_type == "sframe_field":
            frame_father = row_frame['father']
            print(f'\fseq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>20} | section: {frame_section:>35} {frame_reference:>30}')
            result = {}                                 

seq:   5 | model:   mage_1 | type:    sframe_field | Father: 1_frame_prefeitura_nf label:      nome_prefeitura | section:                       1 - CABECALHO                     PREFEITURA
seq:   6 | model:   mage_1 | type:    sframe_field | Father: 1_frame_prefeitura_nf label:           secretaria | section:                       1 - CABECALHO                     SECRETARIA
seq:   7 | model:   mage_1 | type:    sframe_field | Father: 1_frame_prefeitura_nf label:     tipo_nota_fiscal | section:                       1 - CABECALHO                    NOTA FISCAL
seq:   9 | model:   mage_1 | type:    sframe_field | Father: 1_frame_dados_nf label:   numero_nota_fiscal | section:                       1 - CABECALHO                Número da Nota:
seq:  10 | model:   mage_1 | type:    sframe_field | Father: 1_frame_dados_nf label:          competencia | section:                       1 - CABECALHO                   Competência:
seq:  11 | model:   mage_1 | type:    sframe_field | Father

### chamadas de funcao

In [ ]:
                    # secao: 1 - CABECALHO
                    #seq =  0 | 1_frame_prefeitura_nf
                    if frame_pesquisa == "1_frame_prefeitura_nf":
                        data_cabecalho = {}
                        f_father = "1_frame_prefeitura_nf"
                        section = "1 - CABECALHO"
                        Texto_extraido = extract_dados_from_frame(model, f_father, section)
                        print(Texto_extraido)
                        
                        
                        
                    #seq =  1 | 1_frame_dados_nf
                    elif frame_pesquisa == "1_frame_dados_nf":
                        data_prestador = {}
                        f_father = "1_frame_dados_nf"
                        section = "1 - CABECALHO"
                        Texto_extraido = extract_dados_from_frame(model, f_father, section)
                        print(Texto_extraido)
                        
                        
                    # secao: 2. PRESTADOR DE SERVIÇO
                    #seq =  2 | 2_frame_cnpj_prestador
                    elif frame_pesquisa == "2_frame_cnpj_prestador":
                        data_prestador = {}
                        f_father = "2_frame_cnpj_prestador"
                        section = "2. PRESTADOR DE SERVIÇO"
                        Texto_extraido = extract_dados_from_frame(model, f_father, section)
                        print(Texto_extraido)
                        
                    #seq =  3 | 2_frame_inscricao_prestador    
                    elif frame_pesquisa == "2_frame_inscricao_prestador":
                        data_prestador = {}
                        f_father = "2_frame_inscricao_prestador"
                        section = "2. PRESTADOR DE SERVIÇO"
                        Texto_extraido = extract_dados_from_frame(model, f_father, section)
                        print(Texto_extraido)       
                    
                    #seq =  4 | 2_frame_dados_prestador    
                    elif frame_pesquisa == "2_frame_dados_prestador":
                        data_prestador = {}
                        f_father = "2_frame_dados_prestador"
                        section = "2. PRESTADOR DE SERVIÇO"
                        Texto_extraido = extract_dados_from_frame(model, f_father, section)
                        print(Texto_extraido)
                           
                           
                    # secao: 3 - TOMADOR       
                    #seq =  5 | 3_frame_cnpj_tomador       
                    elif frame_pesquisa == "3_frame_cnpj_tomador": #TBD
                    
                    #seq =  6 | 3_frame_inscricao_tomador 
                    elif frame_pesquisa == "3_frame_inscricao_tomador":
                        
                        data_tomador = {}
                        f_father = "3_frame_inscricao_tomador"
                        section = "3. TOMADOR DE SERVIÇO"
                        Texto_extraido = extract_dados_from_frame(model, f_father, section)
                        print(Texto_extraido) 
                    
                    #seq =  7 | 3_frame_dados_tomador    
                    elif frame_pesquisa == "3_frame_dados_tomador":
                        
                        data_tomador = {}
                        f_father = "3_frame_dados_tomador"
                        section = "3. TOMADOR DE SERVIÇO"    
                        Texto_extraido = extract_fields_box_geral(model, f_father, section)
                        print(Texto_extraido)
                        
                        
                    # secao: 4 - DESCRIMINACAO DOS SERVIÇOS
                    #seq =  8 | 4_frame_descricao_totais
                    elif frame_pesquisa == "4_frame_descricao_totais":
                        data_servico = {}
                        result = {}
                        f_father = "4_frame_descricao_totais"
                        section = "4. DESCRIMINACAO DOS SERVIÇOS" 
                        vx_0 = 125
                        vy_0 = 1123
                        vx_1 = 1934
                        vy_1 = 1720
                        data_servico['secao'] = section
                        result, texto_extraido = ocr_RasterPDF_free(image_2work, vx_0, vy_0, vx_1, vy_1)
                        nf_data_servico = {}
                        label = "DISCRIMINAÇÃO DOS SERVIÇOS"
                        if texto_extraido.startswith(label):
                            text = texto_extraido[len(label):].strip()
                        data_servico['discriminacao_servicos'] = text 
                        print(data_servico)
                        
                        
                    # secao: 5. VALOR TOTAL
                    #seq =  9 | 4_frame_valor_total   
                    elif frame_pesquisa == "4_frame_valor_total":
                        data_CNAE = {}
                        f_father = "4_frame_valor_total"
                        section = "5. VALOR TOTAL"    
                        Texto_extraido = extract_fields_box_geral(model, f_father, section)
                        print(Texto_extraido) 
                        
                        
                    # secao: 6. CNAE e Item da Lista de Serviços
                    #seq = 10 | 4_frame_cnae_itens_servico     
                    elif frame_pesquisa == "4_frame_cnae_itens_servico":
                        data_CNAE = {}
                        f_father = "4_frame_cnae_itens_servico"
                        section = "6. CNAE e Item da Lista de Serviços"    
                        Texto_extraido = extract_fields_box(model, f_father, section)
                        
                    
                    # secao: 7. VALORES E IMPOSTOS
                    #seq = 11 | 5_frame_valores_impostos  
                    elif frame_pesquisa == "5_frame_valores_impostos":
                        data_valores = {}
                        result = {}
                        father_value = "5_frame_valores_impostos"
                        section = "7. VALORES E IMPOSTOS"

                        result = extract_fields_box(modelo, father_value, section)
                        if result:
                            data_valores.update(result)
                         

                    #seq = 12 | 5_frame_dados_complementares
                    elif frame_pesquisa == "5_frame_dados_complementares": #TDB
                        
                    
                    #seq = 13 | 5_frame_inf_criticas
                    elif frame_pesquisa == "5_frame_inf_criticas": #TDB    
                    
                    
                    # secao: 10. OBSERVACOES  
                    #seq = 14 | 5_frame_observacao  
                    elif frame_pesquisa == "5_frame_observacao":
                        data_observacao = {}
                        f_father = "5_frame_observacao"
                        section = "10. OBSERVACOES"
                        Texto_extraido = extract_dados_from_frame(model, f_father, section)
                        Texto_extraido

### Tratamento multiplas paginas

In [ ]:
list_document_pages = []

nome_documento = "pipeline_extracao_documentos/2_documentos_para_extracao/Batch_9/1_5148126179567338367.pdf"

path_destino = "pipeline_extracao_documentos/2_documentos_para_extracao/Batch_9"

list_document_pages = simple_page_split(nome_documento)

## Processamento do Pipeline 

In [ ]:
image_resized_path

In [ ]:
root_doc_analise = "pipeline_extracao_documentos/2_documentos_para_extracao/Batch_RASTER_PDF_0"

In [ ]:
root_doc_analise

In [21]:
# 1. Leitura recursiva de diretorios e arquivos a partir de root
pdf_info = {}  # Dicionário para armazenar informações sobre PDFs

nf_data_servico = {}#VERIFICAR
analise_doc_nf = {} #VERIFICAR
file_data = [] #VERIFICAR

list_document_pages = []
#nro_nota = 0
# TEMP
# Nome do arquivo json
nome_formado_json = batch_name +".json"
#3. path formado para nome do arquivo json
json_file_path = os.path.join(json_path, nome_formado_json)
#root_doc_analise = os.path.join(documentos_extracao_path, batch_name)
#print(root_doc_analise)
i = 1
for root, dirs, files in os.walk(root_doc_analise):
    dir_name = os.path.basename(root)
    #print(dir_name)
    for file in files:
        
        if file.lower().endswith('.pdf'):
            doc2convert = file
            document_path_1 = os.path.join(root, file)
            pdf_document = fitz.open(document_path_1)
            #page_number = 0  # Defina o número da página que deseja analisar
            #page = pdf_document[page_number]
            
            documento_pdf = True
            pesquisavel, metadados, paginas = is_pdf_searchable_analise(document_path_1)
            #print(f'\nTeste nro: {i} | doc: {file} | pdf?: {documento_pdf} | pesquisavel?: {pesquisavel} | paginas: {paginas}\n')
                
            if not pesquisavel:
                
                image_2work, name_image_2work = convertResizeAnalise_1page(file, document_path_1, image_resized_path)
                
                nro_nota = 0
                # Frame para teste
                i_frame = 0
                frames_pesquisa = []
                filtered_frames_nf_v4_df = frames_nf_v4_df[frames_nf_v4_df['model'] == model]
                for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
                    frame_seq = row_frame['seq']
                    frame_model = row_frame['model']
                    frame_label = row_frame['label']
                    frame_type = row_frame['type']
                    frame_section = row_frame['section_json']
                    frame_reference = row_frame['reference']
                    frame_father = row_frame['father']
                    if frame_section == "1 - CABECALHO" and frame_type == "frame":
                        data_cabecalho = {}
                        data_cabecalho['secao'] = frame_section
                        if frame_father == "1_frame_prefeitura_nf" and frame_type == "sframe_field":
                        #if frame_label == "1_frame_prefeitura_nf":
                            frame_father = row_frame['father']
                            print(f'\fseq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>20} | section: {frame_section:>35} {frame_reference:>30}')
                            result = {}
                            data_from_pref = {}
                            #Texto_extraido = extract_dados_from_frame(frame_model, frame_father, frame_section)
                        #     result = extract_prefeitura(frame_label, frame_model, Texto_extraido)
                        #     if result:
                        #             data_from_pref.update(result)
                                    
                        #     if data_from_pref:
                        #         data_cabecalho.update(data_from_pref) 
                        # elif frame_label == "1_frame_dados_nf":
                        #     Texto_extraido = extract_dados_from_frame(frame_model, frame_label, frame_section)
                                        
                

                                
                    #print(Texto_extraido, f_father, section, data_from_frame)

                    # #seq = 11 | 5_frame_valores_impostos  
                    # elif frame_pesquisa == "5_frame_valores_impostos":
                    #     data_valores = {}
                    #     result = {}
                    #     father_value = "5_frame_valores_impostos"
                    #     section = "7. VALORES E IMPOSTOS"

                    #     result = extract_fields_box(modelo, father_value, section)
                    #     if result:
                    #         data_from_frame.update(result)

                nome_arquivo = file
                #pdf_info["diretorio"] = os.path.basename(root)
                try:
                    pdf_info[nro_nota] = {
                    "dados_NF_PDF": {
                        "data_cabecalho": data_cabecalho,
                        # "data_prestador": nf_data_prestador,
                        # "data_tomador": nf_data_tomador,
                        # "data_servico": nf_data_servico,
                        # "data_valor_total": nf_data_valor_total,
                        # "data_CNAE": nf_data_CNAE,
                        # "data_valores": nf_data_valores,
                        # "data_dados_complementares": nf_data_dados_complementares,
                        # "data_outras_informacoes": nf_data_outras_informacoes,
                        # "data_observacao": nf_data_observacao,
                    },
                    "diretorio": dir_name, #os.path.basename(root)
                    "nome_arquivo": nome_arquivo,
                    "Batch": batch_name,
                }
                except Exception as e:
                    print(f"Erro ao gerar o json: {e}")
                
                
                pdf_document.close()
                
                #print(pdf_info)




                #if paginas == 1:
                if paginas > 1000:

                        
                    if i == 1000: #Define quantidade de tratamento de documentos raster PDF
                        break

            i +=1 
              
            # 2. Tratamento PDF pesquisavel        
            #else:
                # #print(f'\nPDF PESQUISAVEL nro: {i} | doc: {file} | pdf?: {documento_pdf} | pesquisavel?: {pesquisavel} | paginas: {paginas}\n')
                # status = "O PDF é pesquisável"
                # # Carregar o arquivo PDF
                # pdf_document = fitz.open(file_path)
                
                
                
                # # Página do PDF  ATENCAO  (UNICA PAGINA)
                # page_number = 0  # Defina o número da página que deseja analisar
                # page = pdf_document[page_number]

                
                
                
                
                
                # # Definir retângulo de interesse
                # x0 = 0
                # y0 = 4
                # x1 = 600
                # y1 = 200  # Ajuste este valor para delimitar a região vertical

                # # Extrair texto dentro do retângulo
                # text = page.get_text("text", clip=(x0, y0, x1, y1))

                # if text:
                #    page_number = 0
                # else:
                #    page_number = 1
                
                
                # # 1 - cabecalho
                # #pdf_document = fitz.open(file_path)
                # #page_number = 0  # Defina o número da página que deseja analisar
                # page = pdf_document[page_number]
                # x0 = 0
                # y0 = 0
                # x1 = 600
                # y1 = 110
                
                # text = page.get_text("text", clip=(x0, y0, x1, y1))
                # nf_data_cabecalho = Extc.extract_fields_cabecalho(text)
                # #nf_data_cabecalho = extract_fields_cabecalho(text)
                
                
                # try:
                #     nro_nota = nf_data_cabecalho['numero_nota_fiscal']
                # except Exception as e:
                #     print(f"Erro ao verificar o PDF: {e}")
                    
                
                
                
                # # 2. PRESTADOR DE SERVIÇO
                # # Definir retângulo de interesse
                # x0 = 0
                # y0 = 100
                # x1 = 600
                # y1 = 236  # Ajuste este valor para delimitar a região vertical

                # # Extrair texto dentro do retângulo
                # text = page.get_text("text", clip=(x0, y0, x1, y1))
                
                # nf_data_prestador = Extc.extract_fields_prestador(text)
                # #nf_data_prestador = extract_fields_prestador(text)
                
                # # 3. TOMADOR DE SERVIÇO
                # # Definir retângulo de interesse
                # x0 = 0
                # y0 = 210
                # x1 = 600
                # y1 = 340  # Ajuste este valor para delimitar a região vertical

                # # Extrair texto dentro do retângulo
                # text = page.get_text("text", clip=(x0, y0, x1, y1))
                
                
                # nf_data_tomador = Extc.extrac.extract_fields_tomador(text)
                # #nf_data_tomador = extract_fields_tomador(text)
                
                
                # # 4. DESCRIMINACAO DOS SERVIÇOS
                # nf_data_servico = {}
                # nf_data_servico['secao'] = "4. DESCRIMINACAO DOS SERVIÇOS"
                # # Definir retângulo de interesse
                # x0 = 0
                # y0 = 330
                # x1 = 600
                # y1 = 500  # Ajuste este valor para delimitar a região vertical

                # # Extrair texto dentro do retângulo
                # text = page.get_text("text", clip=(x0, y0, x1, y1))

                # # Remover quebras de linha e rótulo
                # text = text.replace('\n', ' ')
                # label = "DISCRIMINAÇÃO DOS SERVIÇOS"
                # if text.startswith(label):
                #     text = text[len(label):].strip()

                # # Atribuir texto ao dicionário
                # nf_data_servico['discriminacao_servicos'] = text
                
                
                # # 5. VALOR TOTAL
                # nf_data_valor_total = {}
                # nf_data_valor_total['secao'] = "5. VALOR TOTAL"

                # # Definir retângulo de interesse
                # x0 = 0
                # y0 = 500
                # x1 = 600
                # y1 = 550  # Ajuste este valor para delimitar a região vertical


                # # Extrair texto dentro do retângulo
                # text = page.get_text("text", clip=(x0, y0, x1, y1))

                # # Usar expressão regular para extrair apenas os caracteres numéricos e pontos decimais
                # valor_total_match = re.search(r'R\$ ([\d,.]+)', text)
                # if valor_total_match:
                #     valor_total_sem_formatacao = valor_total_match.group(1).replace('.', '').replace(',', '.')
                #     nf_data_valor_total['valor_total_nota'] = float(valor_total_sem_formatacao)
                    
                    
                
                
                # # 6. CNAE e Item da Lista de Serviços
                # nf_data_CNAE = {}
                # nf_data_CNAE['Secao'] = "6. CNAE e Item da Lista de Serviços"

                # # Definir retângulo de interesse CNAE
                # x0 = 0
                # y0 = 530
                # x1 = 600
                # y1 = 540  # Ajuste este valor para delimitar a região vertical


                # # Extrair texto dentro do retângulo
                # text = page.get_text("text", clip=(x0, y0, x1, y1))


                # # Extrair CNAE
                # nf_data_CNAE_match = re.search(r'CNAE\s+(.+)', text)
                # if nf_data_CNAE_match:
                #     # Remove a primeira ocorrência de "CNAE:"
                #     nf_data_CNAE_str = re.sub(r'^CNAE - ', '', text, count=1)
                #     # Remover quebras de linha
                #     nf_data_CNAE_str = nf_data_CNAE_str.replace('\n', ' ')
                #     nf_data_CNAE['cnae'] = nf_data_CNAE_str
                    
                
                
                # # Item da Lista de Serviços    
                # # Definir retângulo de interesse - Item da Lista de Serviços
                # x0 = 0
                # y0 = 545
                # x1 = 600
                # y1 = 560  # Ajuste este valor para delimitar a região vertical    
                # # Extrair texto dentro do retângulo
                # text = page.get_text("text", clip=(x0, y0, x1, y1))     
                    
                # # Extrair Item da Lista de Serviços
                # nf_item_lista_servicos_match = re.search(r'Item da Lista de Serviços\s+(.+)', text)
                # if nf_item_lista_servicos_match:
                #     nf_item_lista_servicos_str = re.sub(r'^Item da Lista de Serviços - ', '', text, count=1) 
                #     # Remover quebras de linha
                #     #nf_item_lista_servicos_str = nf_item_lista_servicos_str.replace('\n \n', '')
                #     nf_item_lista_servicos_str = nf_item_lista_servicos_str.replace('\n', ' ')
                #     nf_data_CNAE['item_lista_servicos'] = nf_item_lista_servicos_str
                  
                
                # # 7. VALORES E IMPOSTOS
                # # Definir retângulo de interesse
                # x0 = 0
                # y0 = 550
                # x1 = 600
                # y1 = 650  # Ajuste este valor para delimitar a região vertical

                # # Extrair texto dentro do retângulo
                # text = page.get_text("text", clip=(x0, y0, x1, y1))

                # # Extrair campos e valores
                # nf_data_valores = Extc.extract_fields_impostos(text)
                # #nf_data_valores = extract_fields_impostos(text)
                
                # # 8. DADOS COMPLEMENTARES
                # nf_data_dados_complementares = {}
                # nf_data_dados_complementares['secao'] = "8. DADOS COMPLEMENTARES"

                # # Definir retângulo de interesse
                # x0 = 0
                # y0 = 650
                # x1 = 600
                # y1 = 680  # Ajuste este valor para delimitar a região vertical

                # # Extrair texto dentro do retângulo
                # text = page.get_text("text", clip=(x0, y0, x1, y1))
                # # Remove a primeira ocorrência de "Observação:"
                # text = re.sub(r'^DADOS COMPLEMENTARES', '', text, count=1)
                # if text == " ":
                #     text = "NONE"
                #     nf_data_dados_complementares['dados_complementares'] = text.strip()
                # else:    
                #     # Extrair texto dentro do retângulo
                #     nf_data_dados_complementares['dados_complementares'] = text.strip()
                
                
                
                # # 9. OUTRAS INFORMAÇOES / CRITICAS  
                # # Definir retângulo de interesse
                # x0 = 0
                # y0 = 680
                # x1 = 600
                # y1 = 725  # Ajuste este valor para delimitar a região vertical

                # # Extrair texto dentro do retângulo
                # text = page.get_text("text", clip=(x0, y0, x1, y1))

                # # Extrair campos e valores
                # nf_data_outras_informacoes = Extc.extract_fields_outras_info(text)
                # #nf_data_outras_informacoes = extract_fields_outras_info(text)
                
                
                # # 10. OBSERVACOES
                # nf_data_observacao = {}
                # nf_data_observacao['secao'] = "10. OBSERVACOES"
                # # Definir retângulo de interesse
                # x0 = 0
                # y0 = 725
                # x1 = 600
                # y1 = 760  # Ajuste este valor para delimitar a região vertical

                # # Extrair texto dentro do retângulo
                # text = page.get_text("text", clip=(x0, y0, x1, y1))

                # # Remove a primeira ocorrência de "Observação:"
                # text = re.sub(r'^Observação:', '', text, count=1)

                # # Remover quebras de linha
                # text = text.replace('\n', ' ')

                # # Extrair texto dentro do retângulo
                # nf_data_observacao['observacao'] = text.strip()
                
                # nr_nro_nf = nro_nota
                
                # nome_arquivo = file
                # #pdf_info["diretorio"] = os.path.basename(root)
                # try:
                #     pdf_info[nro_nota] = {
                #     "dados_NF_PDF": {
                #         "data_cabecalho": nf_data_cabecalho,
                #         "data_prestador": nf_data_prestador,
                #         "data_tomador": nf_data_tomador,
                #         "data_servico": nf_data_servico,
                #         "data_valor_total": nf_data_valor_total,
                #         "data_CNAE": nf_data_CNAE,
                #         "data_valores": nf_data_valores,
                #         "data_dados_complementares": nf_data_dados_complementares,
                #         "data_outras_informacoes": nf_data_outras_informacoes,
                #         "data_observacao": nf_data_observacao,
                #     },
                #     "diretorio": dir_name, #os.path.basename(root)
                #     "nome_arquivo": nome_arquivo,
                #     "Batch": batch_name,
                # }
                # except Exception as e:
                #     print(f"Erro ao gerar o json: {e}")
                
                
                # pdf_document.close()

# # Salvando as informações em um arquivo JSON (novo formato nome arquivo V2)
# with open(json_file_path, "w", encoding='utf-8') as json_file:
#     json.dump(pdf_info, json_file, ensure_ascii=False, indent=4)

# print(f"As informações foram salvas em {json_file_path}")

#print(pdf_info)  

In [ ]:
data_from_prefeitura

In [ ]:
data_from_frame

In [ ]:
def processa_cabecalho():

    nf_data_cabecalho = {}
    model = "mage_1"
    # Filtrar o DataFrame para incluir apenas linhas onde a coluna "model" oriundo de: modelo
    filtered_frames_info = frames_info[frames_info['model'] == model]

    for index_frame, row_frame in filtered_frames_info.iterrows():
        frame_model = row_frame['model']
        if frame_model == model:
            frame_father = row_frame['label']
            # ... Select specific frames ...
            if frame_father == "1_frame_prefeitura_nf" or frame_father == "1_frame_dados_nf":
                
                # Extrai coordenadas para recorte
                x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
                extracted_text_frame = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
                
                seq = row_frame['seq']
                #print(f'\nLoop 1: {frame_model}, seq.: {seq}, row_frame[label]: {frame_father}\n')
                nf_data_cabecalho['secao'] = "1 - CABECALHO"
                
                for index_field, row_field in sframe_fields_info.iterrows():
                    item_son = row_field['father']
                    frame_father = row_frame['label']
                    if item_son == frame_father:

                        model_value = row_field['model']
                        type_value = row_field['type']
                        label_value = row_field['label']
                        reference_value = row_field['reference']
                        seq_value = row_field['seq']
                        father_value = row_field['father']
                        # ... Select specific fields ...
                        if label_value == "nome_prefeitura" and model_value == frame_model: 
                            #print(f'  Loop 2: {model_value}, seq.: {seq_value}, type_value: {type_value}, label_value: {label_value}')
                            values = extracted_text_frame.split('\n')
                            for value in values:
                                result = process_line(value, reference_value, label_value)
                                if result:
                                    nf_data_cabecalho.update(result)
                                    
                        elif label_value == "secretaria" and model_value == frame_model: 
                            values = extracted_text_frame.split('\n')
                            for value in values:
                                result = process_line(value, reference_value, label_value)
                                if result:
                                    nf_data_cabecalho.update(result)
                                    
                        elif label_value == "tipo_nota_fiscal" and model_value == frame_model:
                            values = extracted_text_frame.split('\n')
                            for value in values:
                                result = process_line(value, reference_value, label_value)
                                if result:
                                    nf_data_cabecalho.update(result)
                        
                        #Extraçao de Dados da NF            
                        elif father_value == "1_frame_dados_nf" and model_value == frame_model:
                            values = extracted_text_frame.split('\n')
                            data_list = [item for item in values if item != '']
                            
                            data_dict = {}
                            for i in range(0, len(data_list), 2):
                                key = data_list[i]
                                value = data_list[i+1]
                                data_dict[key] = value

                            # You can now access the values using the corresponding labels
                            nro_nota_fiscal = data_dict['Número da Nota:']
                            nf_data_cabecalho['numero_nota_fiscal'] = data_dict['Número da Nota:']
                            nf_data_cabecalho['competencia'] = data_dict['Competência:']
                            nf_data_cabecalho['dt_hr_emissao'] = data_dict['ata e Hora da Emissão:']
                            nf_data_cabecalho['codigo_verificacao'] = data_dict['Código Verificação:']
                            
    return nf_data_cabecalho, nro_nota_fiscal 

## Blocos de chamada e checagem

In [ ]:
Texto_extraido

In [ ]:
values = Texto_extraido.split('\n')

In [ ]:
values

In [ ]:
f_father

In [ ]:
def process_line(value, reference, label):
    name_match = re.search(fr'{reference} (.+)', value)
    if name_match:
        extracted_value = reference + " " + name_match.group(1)
        return {label: extracted_value}
    return None

In [ ]:
def extract_prefeitura(father, model, text):
    data_extrated_prefeitura = {}

    filtered_sframe_fields_info = sframe_fields_info[(sframe_fields_info['father'] == f_father) & (sframe_fields_info['model'] == model)]

    for index_sframe, row_sframe in filtered_sframe_fields_info.iterrows():
        label_value = row_sframe['label']
        if label_value == "nome_prefeitura":
            reference_value = row_sframe['reference']
            for value in values:
                result = process_line(value, reference_value, label_value)
                if result:
                    data_extrated_prefeitura.update(result)
        elif label_value == "secretaria":
            reference_value = row_sframe['reference']
            for value in values:
                result = process_line(value, reference_value, label_value)
                if result:
                    data_extrated_prefeitura.update(result) 
        elif label_value == "tipo_nota_fiscal":
            reference_value = row_sframe['reference']  
            for value in values:
                result = process_line(value, reference_value, label_value)
                if result:
                    data_extrated_prefeitura.update(result)
                    
    return data_extrated_prefeitura                                      



In [ ]:
data_sframe_info

In [ ]:
sframe_fields_info

In [ ]:
def pesquisa_texto(texto):
    nome_prefeitura_match = re.search(r'PREFEITURA (.+)', texto)
    if nome_prefeitura_match:
        is_prefeitura = "PREFEITURA " + nome_prefeitura_match.group(1)
        
        return  is_prefeitura
    else:
        raise ValueError("Nao consegui pesquisar")

In [ ]:
data_sframe_info = {}

sframe_fields_info

filtered_sframe_fields_info = sframe_fields_info[(sframe_fields_info['father'] == f_father) & (sframe_fields_info['model'] == model)]
for index_sframe, row_sframe in filtered_sframe_fields_info.iterrows():
    reference = row_sframe['reference']


In [ ]:
nf_data_servico = {}
label = "DISCRIMINAÇÃO DOS SERVIÇOS"
if texto_extraido.startswith(label):
    text = texto_extraido[len(label):].strip()
nf_data_servico['discriminacao_servicos'] = text 
print(nf_data_servico)

In [ ]:
nf_data_servico = {}
label = "DISCRIMINAÇÃO DOS SERVIÇOS"
if texto_extraido.startswith(label):
    text = texto_extraido[len(label):].strip()
nf_data_servico['discriminacao_servicos'] = text 
print(nf_data_servico)

In [ ]:
    print(f'\nseq: {seq} |  label: {label} | reference: {reference} ')
    data_sframe_info['seq'] = seq
    data_sframe_info['label'] = label
    data_sframe_info['reference'] = reference

In [ ]:
seq = row_sframe['seq']
label = row_sframe['label']

In [ ]:
result

In [ ]:
data_servico

In [ ]:
def texto_extraido(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [s.replace(":", "") for s in text_splited]
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    return text_splited

### novissimas funcoes de extracao

In [ ]:
#1. funcao: find_value_after_keyword_out_frame_up
def find_value_after_keyword_out_frame_up(keyword, text_list, default_keyword_list=None):
    try:
        index = text_list.index(keyword)
        # Verifica se o valor seguinte não é outra keyword da lista default_keyword_list
        if text_list[index + 1] not in default_keyword_list:
            return text_list[index + 1]
        else:
            return "Valor não encontrado"  # Ou pode retornar None ou uma string vazia, conforme sua necessidade
    except ValueError:
        if default_keyword_list:
            for default_keyword in default_keyword_list:
                if default_keyword in text_list:
                    # Caso especial para 'Nome/Razão Social:'
                    if keyword == 'Nome/Razão Social:':
                        return text_list[0]
        return "Keyword não encontrada"
    
#2. find_value_after_keyword_out_frame_down  
def find_value_after_keyword_out_frame_down(keyword, text_list, default_keyword_list=None):
    try:
        index = text_list.index(keyword)
        # Verifica se o índice seguinte está dentro da lista
        if index + 1 < len(text_list):
            # Verifica se o valor seguinte não é outra keyword da lista default_keyword_list
            if text_list[index + 1] not in default_keyword_list:
                return text_list[index + 1]
            else:
                return "Valor não encontrado"
        else:
            return "Keyword é o último elemento, valor não encontrado"
    except ValueError:
        if default_keyword_list:
            try:
                index = text_list.index(default_keyword_list[-1])
                return text_list[index - 1]
            except ValueError:
                return "Keyword não encontrada"
        else:
            return "Keyword não encontrada"
        
#3. find_value_after_keyword_fuzz
def find_value_after_keyword_fuzz(keyword, text_list, default_keyword_list=None, fuzziness_threshold=80):
    closest_match = None
    closest_match_score = 0
    
    for i, text in enumerate(text_list):
        score = fuzz.ratio(keyword, text)
        
        if score > closest_match_score:
            closest_match_score = score
            closest_match = text
        
        if closest_match_score > fuzziness_threshold:
            break

    if closest_match_score > fuzziness_threshold:
        index = text_list.index(closest_match)
        if index + 1 < len(text_list):
            if text_list[index + 1] not in default_keyword_list:
                return text_list[index + 1]
            else:
                return "Valor não encontrado"
        else:
            return "Keyword é o último elemento, valor não encontrado"
    else:
        return "Keyword não encontrada"            

<mark> Relaçao de Frames para funcoes </mar>

seq =  0 | 1_frame_prefeitura_nf

seq =  1 | 1_frame_dados_nf

seq =  2 | 2_frame_cnpj_prestador

seq =  3 | 2_frame_inscricao_prestador

seq =  4 | 2_frame_dados_prestador

seq =  5 | 3_frame_cnpj_tomador

seq =  6 | 3_frame_inscricao_tomador

seq =  7 | 3_frame_dados_tomador

seq =  8 | 4_frame_descricao_totais

seq =  9 | 4_frame_valor_total

seq = 10 | 4_frame_cnae_itens_servico

seq = 11 | 5_frame_valores_impostos

seq = 12 | 5_frame_dados_complementares

seq = 13 | 5_frame_inf_criticas

seq = 14 | 5_frame_observacao

In [ ]:
# 5. Interacao para pesquisar prefeitura
def pesquisa_texto(texto):
    nome_prefeitura_match = re.search(r'PREFEITURA (.+)', texto)
    if nome_prefeitura_match:
        is_prefeitura = "PREFEITURA " + nome_prefeitura_match.group(1)
        
        return  is_prefeitura
    else:
        raise ValueError("Nao consegui pesquisar")

In [ ]:
keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

### frame_pesquisa = "1_frame_dados_nf"

In [ ]:
text_splited = texto_extraido()

In [ ]:
#5. Itero sobre text_splited (lista)
a = 0
for texto_spl in text_splited:
    print(f'texto_spl linha {a}: {texto_spl}')

    a += 1

In [ ]:
keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

In [ ]:
string_pesquisa = "Número da Nota:"


resultado_extraido_fuzz = find_value_after_keyword_fuzz(string_pesquisa, text_splited, keyword_list)
if resultado_extraido_fuzz == "Keyword não encontrada":
    resultado_extraido_frame_up = find_value_after_keyword_out_frame_up(string_pesquisa, text_splited, keyword_list)
    if resultado_extraido_frame_up == "Keyword não encontrada":
        resultado_extraido_frame_down = find_value_after_keyword_out_frame_down(string_pesquisa, text_splited, keyword_list)
        resultado_extraido = resultado_extraido_frame_down
    else:
        resultado_extraido = resultado_extraido_frame_up
else:
    resultado_extraido = resultado_extraido_fuzz           
    
print(f'"{resultado_extraido}"')

In [ ]:
string_pesquisa = "Competência:"


resultado_extraido_fuzz = find_value_after_keyword_fuzz(string_pesquisa, text_splited, keyword_list)
if resultado_extraido_fuzz == "Keyword não encontrada":
    resultado_extraido_frame_up = find_value_after_keyword_out_frame_up(string_pesquisa, text_splited, keyword_list)
    if resultado_extraido_frame_up == "Keyword não encontrada":
        resultado_extraido_frame_down = find_value_after_keyword_out_frame_down(string_pesquisa, text_splited, keyword_list)
        resultado_extraido = resultado_extraido_frame_down
    else:
        resultado_extraido = resultado_extraido_frame_up
else:
    resultado_extraido = resultado_extraido_fuzz           
    
print(f'"{resultado_extraido}"')

In [ ]:
string_pesquisa = "Código Verificação:"


resultado_extraido_fuzz = find_value_after_keyword_fuzz(string_pesquisa, text_splited, keyword_list)
if resultado_extraido_fuzz == "Keyword não encontrada":
    resultado_extraido_frame_up = find_value_after_keyword_out_frame_up(string_pesquisa, text_splited, keyword_list)
    if resultado_extraido_frame_up == "Keyword não encontrada":
        resultado_extraido_frame_down = find_value_after_keyword_out_frame_down(string_pesquisa, text_splited, keyword_list)
        resultado_extraido = resultado_extraido_frame_down
    else:
        resultado_extraido = resultado_extraido_frame_up
else:
    resultado_extraido = resultado_extraido_fuzz           
    
print(f'"{resultado_extraido}"')

In [ ]:
string_pesquisa = "ata e Hora da Emissão:"

resultado_extraido_fuzz = find_value_after_keyword_fuzz(string_pesquisa, text_splited, keyword_list)
if resultado_extraido_fuzz == "Keyword não encontrada":
    resultado_extraido_frame_up = find_value_after_keyword_out_frame_up(string_pesquisa, text_splited, keyword_list)
    if resultado_extraido_frame_up == "Keyword não encontrada":
        resultado_extraido_frame_down = find_value_after_keyword_out_frame_down(string_pesquisa, text_splited, keyword_list)
        resultado_extraido = resultado_extraido_frame_down
    else:
        resultado_extraido = resultado_extraido_frame_up
else:
    resultado_extraido = resultado_extraido_fuzz           
    
print(f'"{resultado_extraido}"')

### frame_pesquisa = "2_frame_inscricao_prestador"

In [ ]:
#5. Itero sobre text_splited (lista)
a = 0
for texto_spl in text_splited:
    print(f'texto_spl linha {a}: {texto_spl}')

    a += 1

In [ ]:
keyword_list = ['Inscrição Municipal:', 'Inscrição Estadual:']

In [ ]:
string_pesquisa = "Inscrição Municipal:"


resultado_extraido_fuzz = find_value_after_keyword_fuzz(string_pesquisa, text_splited, keyword_list)
if resultado_extraido_fuzz == "Keyword não encontrada":
    resultado_extraido_frame_up = find_value_after_keyword_out_frame_up(string_pesquisa, text_splited, keyword_list)
    if resultado_extraido_frame_up == "Keyword não encontrada":
        resultado_extraido_frame_down = find_value_after_keyword_out_frame_down(string_pesquisa, text_splited, keyword_list)
        resultado_extraido = resultado_extraido_frame_down
    else:
        resultado_extraido = resultado_extraido_frame_up
else:
    resultado_extraido = resultado_extraido_fuzz           
    
print(f'"{resultado_extraido}"')

In [ ]:
string_pesquisa = "Inscrição Estadual:"


resultado_extraido_fuzz = find_value_after_keyword_fuzz(string_pesquisa, text_splited, keyword_list)
if resultado_extraido_fuzz == "Keyword não encontrada":
    resultado_extraido_frame_up = find_value_after_keyword_out_frame_up(string_pesquisa, text_splited, keyword_list)
    if resultado_extraido_frame_up == "Keyword não encontrada":
        resultado_extraido_frame_down = find_value_after_keyword_out_frame_down(string_pesquisa, text_splited, keyword_list)
        resultado_extraido = resultado_extraido_frame_down
    else:
        resultado_extraido = resultado_extraido_frame_up
else:
    resultado_extraido = resultado_extraido_fuzz           
    
print(f'"{resultado_extraido}"')

### frame_pesquisa = "2_frame_dados_prestador"

In [ ]:
#5. Itero sobre text_splited (lista)
a = 0
for texto_spl in text_splited:
    print(f'texto_spl linha {a}: {texto_spl}')

    a +=1

In [ ]:
keyword_list = ['Nome/Razão Social:', 'Nome de Fantasia:', 'Endereço:', 'E-mail:']

string_pesquisa = "Nome/Razão Social:"


resultado_extraido_fuzz = find_value_after_keyword_fuzz(string_pesquisa, text_splited, keyword_list)
if resultado_extraido_fuzz == "Keyword não encontrada":
    resultado_extraido_frame_up = find_value_after_keyword_out_frame_up(string_pesquisa, text_splited, keyword_list)
    if resultado_extraido_frame_up == "Keyword não encontrada":
        resultado_extraido_frame_down = find_value_after_keyword_out_frame_down(string_pesquisa, text_splited, keyword_list)
        resultado_extraido = resultado_extraido_frame_down
    else:
        resultado_extraido = resultado_extraido_frame_up
else:
    resultado_extraido = resultado_extraido_fuzz           
    
print(f'"{resultado_extraido}"')
    



In [ ]:
keyword_list = ['Nome/Razão Social:', 'Nome de Fantasia:', 'Endereço:', 'E-mail:']

string_pesquisa = "Nome de Fantasia:"


resultado_extraido_fuzz = find_value_after_keyword_fuzz(string_pesquisa, text_splited, keyword_list)
if resultado_extraido_fuzz == "Keyword não encontrada":
    resultado_extraido_frame_up = find_value_after_keyword_out_frame_up(string_pesquisa, text_splited, keyword_list)
    if resultado_extraido_frame_up == "Keyword não encontrada":
        resultado_extraido_frame_down = find_value_after_keyword_out_frame_down(string_pesquisa, text_splited, keyword_list)
        resultado_extraido = resultado_extraido_frame_down
    else:
        resultado_extraido = resultado_extraido_frame_up
else:
    resultado_extraido = resultado_extraido_fuzz           
    
print(f'"{resultado_extraido}"')

In [ ]:
keyword_list = ['Nome/Razão Social:', 'Nome de Fantasia:', 'Endereço:', 'E-mail:']

string_pesquisa = "Endereço:"


resultado_extraido_fuzz = find_value_after_keyword_fuzz(string_pesquisa, text_splited, keyword_list)
if resultado_extraido_fuzz == "Keyword não encontrada":
    resultado_extraido_frame_up = find_value_after_keyword_out_frame_up(string_pesquisa, text_splited, keyword_list)
    if resultado_extraido_frame_up == "Keyword não encontrada":
        resultado_extraido_frame_down = find_value_after_keyword_out_frame_down(string_pesquisa, text_splited, keyword_list)
        resultado_extraido = resultado_extraido_frame_down
    else:
        resultado_extraido = resultado_extraido_frame_up
else:
    resultado_extraido = resultado_extraido_fuzz           
    
print(f'"{resultado_extraido}"')


In [ ]:
keyword_list = ['Nome/Razão Social:', 'Nome de Fantasia:', 'Endereço:', 'E-mail:']

string_pesquisa = "E-mail:"


resultado_extraido_fuzz = find_value_after_keyword_fuzz(string_pesquisa, text_splited, keyword_list)
if resultado_extraido_fuzz == "Keyword não encontrada":
    resultado_extraido_frame_up = find_value_after_keyword_out_frame_up(string_pesquisa, text_splited, keyword_list)
    if resultado_extraido_frame_up == "Keyword não encontrada":
        resultado_extraido_frame_down = find_value_after_keyword_out_frame_down(string_pesquisa, text_splited, keyword_list)
        resultado_extraido = resultado_extraido_frame_down
    else:
        resultado_extraido = resultado_extraido_frame_up
else:
    resultado_extraido = resultado_extraido_fuzz           
    
print(f'"{resultado_extraido}"')

### frame_pesquisa = "3_frame_inscricao_tomador"

In [ ]:
#5. Itero sobre text_splited (lista)
a = 0
for texto_spl in text_splited:
    print(f'texto_spl linha {a}: {texto_spl}')

    a += 1

In [ ]:
keyword_list = ['RG:', 'Inscrição Estadual:']

In [ ]:
string_pesquisa = "RG:"


resultado_extraido_fuzz = find_value_after_keyword_fuzz(string_pesquisa, text_splited, keyword_list)
if resultado_extraido_fuzz == "Keyword não encontrada":
    resultado_extraido_frame_up = find_value_after_keyword_out_frame_up(string_pesquisa, text_splited, keyword_list)
    if resultado_extraido_frame_up == "Keyword não encontrada":
        resultado_extraido_frame_down = find_value_after_keyword_out_frame_down(string_pesquisa, text_splited, keyword_list)
        resultado_extraido = resultado_extraido_frame_down
    else:
        resultado_extraido = resultado_extraido_frame_up
else:
    resultado_extraido = resultado_extraido_fuzz           
    
print(f'"{resultado_extraido}"')

In [ ]:
string_pesquisa = "Inscrição Estadual:"


resultado_extraido_fuzz = find_value_after_keyword_fuzz(string_pesquisa, text_splited, keyword_list)
if resultado_extraido_fuzz == "Keyword não encontrada":
    resultado_extraido_frame_up = find_value_after_keyword_out_frame_up(string_pesquisa, text_splited, keyword_list)
    if resultado_extraido_frame_up == "Keyword não encontrada":
        resultado_extraido_frame_down = find_value_after_keyword_out_frame_down(string_pesquisa, text_splited, keyword_list)
        resultado_extraido = resultado_extraido_frame_down
    else:
        resultado_extraido = resultado_extraido_frame_up
else:
    resultado_extraido = resultado_extraido_fuzz           
    
print(f'"{resultado_extraido}"')

### frame_pesquisa ="3_frame_dados_tomador"

In [ ]:
#5. Itero sobre text_splited (lista)
a = 0
for texto_spl in text_splited:
    print(f'texto_spl linha {a}: {texto_spl}')

    a +=1

In [ ]:
keyword_list = ['Nome/Razão Social:', 'Endereço:', 'E-mail']

string_pesquisa = "Nome/Razão Social:"


resultado_extraido_fuzz = find_value_after_keyword_fuzz(string_pesquisa, text_splited, keyword_list)
if resultado_extraido_fuzz == "Keyword não encontrada":
    resultado_extraido_frame_up = find_value_after_keyword_out_frame_up(string_pesquisa, text_splited, keyword_list)
    if resultado_extraido_frame_up == "Keyword não encontrada":
        resultado_extraido_frame_down = find_value_after_keyword_out_frame_down(string_pesquisa, text_splited, keyword_list)
        resultado_extraido = resultado_extraido_frame_down
    else:
        resultado_extraido = resultado_extraido_frame_up
else:
    resultado_extraido = resultado_extraido_fuzz           
    
print(f'"{resultado_extraido}"')
    

In [ ]:
keyword_list = ['Nome/Razão Social:', 'Endereço:', 'E-mail']

string_pesquisa = "Endereço:"


resultado_extraido_fuzz = find_value_after_keyword_fuzz(string_pesquisa, text_splited, keyword_list)
if resultado_extraido_fuzz == "Keyword não encontrada":
    resultado_extraido_frame_up = find_value_after_keyword_out_frame_up(string_pesquisa, text_splited, keyword_list)
    if resultado_extraido_frame_up == "Keyword não encontrada":
        resultado_extraido_frame_down = find_value_after_keyword_out_frame_down(string_pesquisa, text_splited, keyword_list)
        resultado_extraido = resultado_extraido_frame_down
    else:
        resultado_extraido = resultado_extraido_frame_up
else:
    resultado_extraido = resultado_extraido_fuzz           
    
print(f'"{resultado_extraido}"')

In [ ]:
keyword_list = ['Nome/Razão Social:', 'Endereço:', 'E-mail']

string_pesquisa = "E-mail"


resultado_extraido_fuzz = find_value_after_keyword_fuzz(string_pesquisa, text_splited, keyword_list)
if resultado_extraido_fuzz == "Keyword não encontrada":
    resultado_extraido_frame_up = find_value_after_keyword_out_frame_up(string_pesquisa, text_splited, keyword_list)
    if resultado_extraido_frame_up == "Keyword não encontrada":
        resultado_extraido_frame_down = find_value_after_keyword_out_frame_down(string_pesquisa, text_splited, keyword_list)
        resultado_extraido = resultado_extraido_frame_down
    else:
        resultado_extraido = resultado_extraido_frame_up
else:
    resultado_extraido = resultado_extraido_fuzz           
    
print(f'"{resultado_extraido}"')

### frame_pesquisa = "4_frame_descricao_totais"

In [ ]:
nf_data_servico = {}
label = "DISCRIMINAÇÃO DOS SERVIÇOS"
if texto_extraido.startswith(label):
    text = texto_extraido[len(label):].strip()
nf_data_servico['discriminacao_servicos'] = text 
print(nf_data_servico)

In [ ]:
print(nf_data_servico)

### frame_pesquisa = "4_frame_valor_total"

In [ ]:
Texto_extraido

In [ ]:
match = re.search(r'R\$\s[\d,.]+', Texto_extraido)

if match:
    extracted_value = match.group(0)
else:
    extracted_value = "Valor não encontrado"

valor_total = format_number(extracted_value)

print(valor_total)

In [ ]:
     # Filtrar o DataFrame para incluir apenas linhas onde a coluna "model" oriundo de: modelo
                filtered_frames_info = frames_info[frames_info['model'] == modelo]
                for index_frame, row_frame in filtered_frames_info.iterrows():
                    frame_name = row_frame['label']
                    frames_pesquisa.append(frame_name)
                    data_cabecalho = {}
                    section = "1 - CABECALHO"
                    data_frame = "1_frame_prefeitura_nf"
                    if frames_pesquisa[i_frame] == data_frame:
                        print(f'\nTeste nro: {i} | doc: {file} | pdf?: {documento_pdf} | pesquisavel?: {pesquisavel} | paginas: {paginas}\n')
                        result = {}
                        data_from_pref = {}
                        f_father = frames_pesquisa[i_frame]
                        Texto_extraido = extract_dados_from_frame(model, f_father, section)
                        result = extract_prefeitura(f_father, model, Texto_extraido)
                        if result:
                                data_from_pref.update(result)
                                
                        if data_from_pref:
                            data_cabecalho.update(data_from_pref)      
                            
                            
                elif frames_pesquisa[i_frame] == data_frame:
                    print(f'\nTeste nro: {i} | doc: {file} | pdf?: {documento_pdf} | pesquisavel?: {pesquisavel} | paginas: {paginas}\n')
                    data_frame = "1_frame_dados_nf"            
                    f_father = frames_pesquisa[i_frame]
                    Texto_extraido = extract_dados_from_frame(model, f_father, section)
                    #result = extract_prefeitura(f_father, model, Texto_extraido)

### frame_pesquisa = "4_frame_cnae_itens_servico"

In [ ]:

mage_cnae_x_item_servico_df = pd.read_excel('pipeline_extracao_documentos/6_geral_administacao/datasets/MAGE_CNAE_X_ITEM_SERVICO_V1.xlsx')

# Creating a dictionary for CNAE codes and descriptions
cnae_dict = dict(zip(mage_cnae_x_item_servico_df['cnae'], mage_cnae_x_item_servico_df['descricao_cnae']))
item_servico_dict = dict(zip(mage_cnae_x_item_servico_df['item_servico'], mage_cnae_x_item_servico_df['descricao_item_servico']))



# Função para extrair número da string
def extract_number(text):
    match = re.search(r'\b\d+(\.\d+)?\b', text)
    if match:
        return match.group(0)
    else:
        return None

In [ ]:
Texto_extraido
text_splited = Texto_extraido.split('\n')

In [ ]:
# Processando CNAE
cnae_line = [line for line in text_splited if 'CNAE' in line][0]
cnae_number = int(extract_number(cnae_line))
cnae_value = cnae_dict.get(cnae_number, "Valor não encontrado")
cnae_value = cnae_value.upper()
cnae_value = str(cnae_number) + " - " + cnae_value
print(cnae_value)

In [ ]:
# Processando Item de Serviço
item_servico_line = [line for line in text_splited if 'Item da Lista de Serviços' in line][0]
item_servico_number = float(extract_number(item_servico_line))
item_servico_value = item_servico_dict.get(item_servico_number, "Valor não encontrado")
item_servico_value = item_servico_value.upper()
item_servico_value = str(item_servico_number) + " - " + item_servico_value
print(item_servico_value)

## Desenha Box

In [ ]:
name_image_2work = "pipeline_extracao_documentos/6_geral_administacao/images/processadas/Doria Marinho 0297 Raquel.pdf.jpg"

In [ ]:
name_image_2work = "pipeline_extracao_documentos/6_geral_administacao/images/processadas/Doria Marinho 0299 Luciana.pdf.jpg"

In [ ]:
# Define a mapping for color names to RGB values
color_mapping = {
    "red": (255, 0, 0),
    "purple": (128, 0, 128),
    "orange": (255, 165, 0),
    "green": (0, 128, 50),
    "blue": (0, 0, 255),
    "yellow": (255, 255, 0)
}

# Reload the image to start fresh
image = Image.open(name_image_2work)
draw = ImageDraw.Draw(image)

# Define a font size for the labels using the default PIL font
font_size = 100
#font = ImageFont.load_default()

font = ImageFont.truetype("/usr/share/fonts/truetype/ubuntu/Ubuntu-M.ttf", 30, encoding="unic")

# Update the draw_box function to use the larger font size with the default font
def draw_box(row):
    x0, y0, x1, y1 = row['x0'], row['y0'], row['x1'], row['y1']
    color = color_mapping.get(row['color'], (0, 0, 0)) # Default to black if color not found
    draw.rectangle([x0, y0, x1, y1], outline=color, width=3)
    label = str(row['label']) if pd.notnull(row['label']) else None # Check for missing label
    if label:
        draw.text((x0 + 5, y0 + 5), label, fill=color, font=font)

# Draw the boundaries
#draw_box(boundaries_info)


def draw_box_model(modelo,
                   boundaries_info=None,
                   sections_info=None,
                   frames_info=None,
                   field_boxes_info=None,
                   draw_boundaries=True,
                   draw_sections=True,
                   draw_frames=True,
                   draw_field_boxes=True):
    
    # Draw boundaries if requested
    if draw_boundaries and boundaries_info is not None:
        filtered_boundaries_info = boundaries_info[boundaries_info['model'] == modelo]
        for index, row in filtered_boundaries_info.iterrows():
            draw_box(row)

    # Draw sections if requested
    if draw_sections and sections_info is not None:
        filtered_sections_info = sections_info[sections_info['model'] == modelo]
        for index, row in filtered_sections_info.iterrows():
            draw_box(row)
            
    # Draw frames if requested
    if draw_frames and frames_info is not None:
        filtered_frames_info = frames_info[frames_info['model'] == modelo]
        for index, row in filtered_frames_info.iterrows():
            draw_box(row)
            
    # Draw field boxes if requested
    if draw_field_boxes and field_boxes_info is not None:
        filtered_field_boxes_info = field_boxes_info[field_boxes_info['model'] == modelo]
        for index, row in filtered_field_boxes_info.iterrows():
            draw_box(row)
    
    # Show the image with selected drawings
    image.show()

In [ ]:
modelo = "mage_1"

In [ ]:
# To draw everything
draw_box_model(modelo, boundaries_info, sections_info, frames_info, field_boxes_info)


In [ ]:
# To draw only boundaries and sections:
draw_box_model(modelo, boundaries_info, sections_info, draw_frames=False, draw_field_boxes=False)


In [ ]:
# To draw only field boxes:
draw_box_model(modelo, field_boxes_info=field_boxes_info, draw_boundaries=False, draw_sections=False, draw_frames=False)


# Outros

### Frames de pesquisa

In [ ]:
                    # secao: 1 - CABECALHO
                    #seq =  0 | 1_frame_prefeitura_nf
                    if frame_pesquisa == "1_frame_prefeitura_nf":
                        data_cabecalho = {}
                        f_father = "1_frame_prefeitura_nf"
                        section = "1 - CABECALHO"
                        Texto_extraido = extract_dados_from_frame(model, f_father, section)
                        print(Texto_extraido)
                    #seq =  1 | 1_frame_dados_nf
                    elif frame_pesquisa == "1_frame_dados_nf":
                        data_prestador = {}
                        f_father = "1_frame_dados_nf"
                        section = "1 - CABECALHO"
                        Texto_extraido = extract_dados_from_frame(model, f_father, section)
                        print(Texto_extraido)
                        
                        
                    # secao: 2. PRESTADOR DE SERVIÇO
                    #seq =  2 | 2_frame_cnpj_prestador
                    elif frame_pesquisa == "2_frame_cnpj_prestador":
                        data_prestador = {}
                        f_father = "2_frame_cnpj_prestador"
                        section = "2. PRESTADOR DE SERVIÇO"
                        Texto_extraido = extract_dados_from_frame(model, f_father, section)
                        print(Texto_extraido)
                        
                    #seq =  3 | 2_frame_inscricao_prestador    
                    elif frame_pesquisa == "2_frame_inscricao_prestador":
                        data_prestador = {}
                        f_father = "2_frame_inscricao_prestador"
                        section = "2. PRESTADOR DE SERVIÇO"
                        Texto_extraido = extract_dados_from_frame(model, f_father, section)
                        print(Texto_extraido)       
                    
                    #seq =  4 | 2_frame_dados_prestador    
                    elif frame_pesquisa == "2_frame_dados_prestador":
                        data_prestador = {}
                        f_father = "2_frame_dados_prestador"
                        section = "2. PRESTADOR DE SERVIÇO"
                        Texto_extraido = extract_dados_from_frame(model, f_father, section)
                        print(Texto_extraido)
                           
                           
                    # secao: 3 - TOMADOR       
                    #seq =  5 | 3_frame_cnpj_tomador       
                    elif frame_pesquisa == "3_frame_cnpj_tomador": #TBD
                    
                    #seq =  6 | 3_frame_inscricao_tomador 
                    elif frame_pesquisa == "3_frame_inscricao_tomador":
                        
                        data_tomador = {}
                        f_father = "3_frame_inscricao_tomador"
                        section = "3. TOMADOR DE SERVIÇO"
                        Texto_extraido = extract_dados_from_frame(model, f_father, section)
                        print(Texto_extraido) 
                    
                    #seq =  7 | 3_frame_dados_tomador    
                    elif frame_pesquisa == "3_frame_dados_tomador":
                        
                        data_tomador = {}
                        f_father = "3_frame_dados_tomador"
                        section = "3. TOMADOR DE SERVIÇO"    
                        Texto_extraido = extract_fields_box_geral(model, f_father, section)
                        print(Texto_extraido)
                        
                        
                    # secao: 4 - DESCRIMINACAO DOS SERVIÇOS
                    #seq =  8 | 4_frame_descricao_totais
                    elif frame_pesquisa == "4_frame_descricao_totais":
                        data_servico = {}
                        result = {}
                        f_father = "4_frame_descricao_totais"
                        section = "4. DESCRIMINACAO DOS SERVIÇOS" 
                        vx_0 = 125
                        vy_0 = 1123
                        vx_1 = 1934
                        vy_1 = 1720
                        data_servico['secao'] = section
                        result, texto_extraido = ocr_RasterPDF_free(image_2work, vx_0, vy_0, vx_1, vy_1)
                        nf_data_servico = {}
                        label = "DISCRIMINAÇÃO DOS SERVIÇOS"
                        if texto_extraido.startswith(label):
                            text = texto_extraido[len(label):].strip()
                        data_servico['discriminacao_servicos'] = text 
                        print(data_servico)
                        
                        
                    # secao: 5. VALOR TOTAL
                    #seq =  9 | 4_frame_valor_total   
                    elif frame_pesquisa == "4_frame_valor_total":
                        data_CNAE = {}
                        f_father = "4_frame_valor_total"
                        section = "5. VALOR TOTAL"    
                        Texto_extraido = extract_fields_box_geral(model, f_father, section)
                        print(Texto_extraido) 
                        
                        
                    # secao: 6. CNAE e Item da Lista de Serviços
                    #seq = 10 | 4_frame_cnae_itens_servico     
                    elif frame_pesquisa == "4_frame_cnae_itens_servico":
                        data_CNAE = {}
                        f_father = "4_frame_cnae_itens_servico"
                        section = "6. CNAE e Item da Lista de Serviços"    
                        Texto_extraido = extract_fields_box(model, f_father, section)
                        
                    
                    # secao: 7. VALORES E IMPOSTOS
                    #seq = 11 | 5_frame_valores_impostos  
                    elif frame_pesquisa == "5_frame_valores_impostos":
                        data_valores = {}
                        result = {}
                        father_value = "5_frame_valores_impostos"
                        section = "7. VALORES E IMPOSTOS"

                        result = extract_fields_box(modelo, father_value, section)
                        if result:
                            data_valores.update(result)
                         

                    #seq = 12 | 5_frame_dados_complementares
                    elif frame_pesquisa == "5_frame_dados_complementares": #TDB
                        
                    
                    #seq = 13 | 5_frame_inf_criticas
                    elif frame_pesquisa == "5_frame_inf_criticas": #TDB    
                    
                    
                    # secao: 10. OBSERVACOES  
                    #seq = 14 | 5_frame_observacao  
                    elif frame_pesquisa == "5_frame_observacao":
                        data_observacao = {}
                        f_father = "5_frame_observacao"
                        section = "10. OBSERVACOES"
                        Texto_extraido = extract_dados_from_frame(model, f_father, section)
                        Texto_extraido

### Modelo de funcoes de extracao - PDF Pesquisavel

In [ ]:
nf_data_servico = {}
# 0. Pesquisa PDF
def is_pdf_searchable(pdf_path):
    try:
        pdf_document = fitz.open(pdf_path)
        is_searchable = all(page.get_text("text") != "" for page in pdf_document)
        pdf_document.close()
        return is_searchable
    except Exception as e:
        print(f"Erro ao verificar o PDF: {e}")
        return False

# 1. CABECALHO
def extract_fields_cabecalho(text):
    nf_data_cabecalho = {}
    nf_data_cabecalho['secao'] = "1 - CABECALHO"
    
    
    # Extrair Nome da Prefeitura
    nome_prefeitura_match = re.search(r'PREFEITURA (.+)', text)
    if nome_prefeitura_match:
        nome_prefeitura = "PREFEITURA " + nome_prefeitura_match.group(1)
        nf_data_cabecalho['nome_prefeitura'] = nome_prefeitura

    # Extrair Tipo de NF
    tipo_nf_match = re.search(r'NOTA FISCAL (.+)', text)
    if tipo_nf_match:
        tipo_nf = "NOTA FISCAL " + tipo_nf_match.group(1)
        nf_data_cabecalho['tipo_nota_fiscal'] = tipo_nf
    
    # Extrair Número da Nota
    numero_nota_match = re.search(r'Número da Nota:\s+(\d+)', text)
    if numero_nota_match:
        nr_nro_nf = numero_nota_match.group(1)
        nf_data_cabecalho['numero_nota_fiscal'] = numero_nota_match.group(1)

    # Extrair Competência
    competencia_match = re.search(r'Competência:\s+(.+)', text)
    if competencia_match:
        nf_data_cabecalho['competencia'] = competencia_match.group(1)

    # Extrair Data e Hora de Emissão
    data_emissao_match = re.search(r'Data e Hora da Emissão:\s+(.+)', text)
    if data_emissao_match:
        nf_data_cabecalho['dt_hr_emissao'] = data_emissao_match.group(1)
        
    # Extrair Data e Hora de Emissão
    codigo_verificacao_match = re.search(r'Código Verificação:\s+(.+)', text)
    if codigo_verificacao_match:
        nf_data_cabecalho['codigo_verificacao'] = codigo_verificacao_match.group(1)    

    return nf_data_cabecalho

# 2. PRESTADOR DE SERVIÇO
def extract_fields_prestador(text): # Função para extrair campos e valores dentro de um retângulo
    nf_data_prestador = {}
    
    nf_data_prestador['secao'] = "2. PRESTADOR DE SERVIÇO"
    
    
    # Extrair CPF/CNPJ com máscara 1
    if "CPF/CNPJ:" in text:
        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', text)
        if cpf_cnpj_formatado_match:
                        nf_data_prestador['cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
                        nf_data_prestador['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))

    # Extrair Inscrição Municipal
    inscricao_municipal_match = re.search(r'Inscrição Municipal:\s+(.+)', text)
    if inscricao_municipal_match:
        nf_data_prestador['inscricao_municipal'] = inscricao_municipal_match.group(1)
        
               
    # Extrair Inscrição Estadual
    #if "Inscrição Estadual:" in text:
    
    # Extrair Inscrição Estadual
    inscricao_estadual_match = re.search(r'Inscrição Estadual:\s+(.+)', text)
    if inscricao_estadual_match:
        inscricao_estadual_str = inscricao_estadual_match.group(1)
        if inscricao_estadual_str == 'Nome/Razão Social:':
            nf_data_prestador['inscricao_estadual'] = "NONE"
        else:    
            nf_data_prestador['inscricao_estadual'] = inscricao_estadual_match.group(1)       
        
                
    

    # Extrair Telefone
    #telefone_match = re.search(r'Telefone:\s+([0-9.\s-])', text)
    telefone_match = re.search(r'Telefone:\s+([0-9.\s-]+)', text)
    if telefone_match: 
        telefone_str = telefone_match.group(1)
        # Remover quebras de linha
        telefone_str = telefone_str.replace('.', '')
        telefone_str = telefone_str.replace('\n', '')
                
        nf_data_prestador['telefone'] = telefone_str
    else:
        nf_data_prestador['telefone'] = "NONE"

         
                
    # Nome/Razão Social:
    razao_social_match = re.search(r'Nome/Razão Social:\s+(.+)', text)
    if razao_social_match:
        nf_data_prestador['razao_social'] = razao_social_match.group(1)  
                
    # Nome de Fantasia:
    nome_fantasia_match = re.search(r'Nome de Fantasia:\s+(.+)', text)
    if nome_fantasia_match:
        nf_data_prestador['nome_fantasia'] = nome_fantasia_match.group(1)                                    
                
            
    # Endereço:
    endereco_match = re.search(r'Endereço:\s+(.+)', text)
    if endereco_match:
        nf_data_prestador['endereco'] = endereco_match.group(1) 
    
    # E-mail:
    email_match = re.search(r'E-mail:\s+(.+)', text)
    if email_match:
        nf_data_prestador['email'] = email_match.group(1)  
    else:
        nf_data_prestador['email'] = "NONE"  # Valor padrão quando não há correspondência
   
        

    return nf_data_prestador

# 3. TOMADOR DE SERVIÇO
def extract_fields_tomador(text):
    nf_data_tomador = {}
    
    
    nf_data_tomador['secao'] = "3. TOMADOR DE SERVIÇO"
    
    
    # Extrair CPF/CNPJ com máscara 1
    if "CPF/CNPJ:" in text:
        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', text)
        if cpf_cnpj_formatado_match:
                        nf_data_tomador['cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
                        nf_data_tomador['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))

        
    # Extrair RG    
    rg_match = re.search(r'RG:\s+(.+)', text)   
    if rg_match:
        rg_str = rg_match.group(1)
        if rg_str == 'Telefone:':
            nf_data_tomador['rg'] = "NONE"  # Valor padrão quando não há correspondência
        else:    
            nf_data_tomador['rg'] = rg_match.group(1)  
 
        
    # Extrair Telefone
    telefone_match = re.search(r'Telefone:\s+(.+)', text)
    if telefone_match:
        telefone_str = telefone_match.group(1)
        if telefone_str == 'Inscrição Estadual:':
            nf_data_tomador['telefone'] = "NONE"  # Valor padrão quando não há correspondência
        else:    
            nf_data_tomador['telefone'] = telefone_match.group(1)
     

    # Extrair Inscrição Municipal
    inscricao_municipal_match = re.search(r'Inscrição Municipal:\s+(.+)', text)
    if inscricao_municipal_match:
        nf_data_tomador['inscricao_municipal'] = inscricao_municipal_match.group(1)
                
                
                
    # Extrair Inscrição Estadual
    inscricao_estadual_match = re.search(r'Inscrição Estadual:\s+(.+)', text)
    if inscricao_estadual_match:
        inscricao_estadual_str = inscricao_estadual_match.group(1)
        if inscricao_estadual_str == 'Nome/Razão Social:':
            nf_data_tomador['inscricao_estadual'] = "NONE"
        else:    
            nf_data_tomador['inscricao_estadual'] = inscricao_estadual_match.group(1)   
                
    
    # Nome/Razão Social:
    razao_social_match = re.search(r'Nome/Razão Social:\s+(.+)', text)
    if razao_social_match:
        nf_data_tomador['razao_social'] = razao_social_match.group(1)                                                
                
    # Endereço:
    endereco_match = re.search(r'Endereço:\s+(.+)', text)
    if endereco_match:
        nf_data_tomador['endereco'] = endereco_match.group(1) 
    
    # E-mail:
    email_match = re.search(r'E-mail:\s+(.+)', text)
    if email_match:
        nf_data_tomador['email'] = email_match.group(1) 
    else:
        nf_data_tomador['email'] = "NONE"  # Valor padrão quando não há correspondência    

    return nf_data_tomador

# 7. VALORES E IMPOSTOS
def extract_fields_impostos(text):
    nf_data_valores = {}
    nf_data_valores['secao'] = "7. VALORES E IMPOSTOS"
    
    # Extrair VALOR SERVIÇOS:
    valor_servicos_match = re.search(r'VALOR SERVIÇOS:\s+(.+)', text)
    if valor_servicos_match:
        valor_servicos_str = valor_servicos_match.group(1)
        valor_servicos_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_servicos_str)
        if valor_servicos_sem_formato:
            valor_servicos_sem_formatacao = valor_servicos_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['valor_servicos'] = float(valor_servicos_sem_formatacao)
        else:
            nf_data_valores['valor_servicos'] = 0.0  # Valor não encontrado ou não está no formato esperado
  
  
    # Extrair VALOR DEDUÇÃO:
    valor_deducao_match = re.search(r'DEDUÇÃO:\s+(.+)', text)
    if valor_deducao_match:
        valor_deducao_str = valor_deducao_match.group(1)
        valor_deducao_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_deducao_str)
        if valor_deducao_sem_formato:
            valor_deducao_sem_formato = valor_deducao_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['valor_deducao'] = float(valor_deducao_sem_formato)
        else:
            nf_data_valores['valor_deducao'] = 0.0  # Valor não encontrado ou não está no formato esperado
        
        
    # Extrair DESC. INCOND:
    valor_desc_match = re.search(r'DESC. INCOND:\s+(.+)', text)
    if valor_desc_match:
        valor_desc_str = valor_desc_match.group(1)
        valor_desc_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_desc_str)
        if valor_desc_sem_formato:
            valor_desc_sem_formato = valor_desc_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['desc_incond'] = float(valor_desc_sem_formato)
        else:
            nf_data_valores['desc_incond'] = 0.0  # Valor não encontrado ou não está no formato esperado        
        

    # Extrair BASE DE CÁLCULO:
    valor_calculo_match = re.search(r'CÁLCULO:\s+(.+)', text)
    if valor_calculo_match:
        valor_calculo_str = valor_calculo_match.group(1)
        valor_calculo_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_calculo_str)
        if valor_calculo_sem_formato:
            valor_calculo_sem_formato = valor_calculo_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['base_calculo'] = float(valor_calculo_sem_formato)
        else:
            nf_data_valores['base_calculo'] = 0.0  # Valor não encontrado ou não está no formato esperado    



    # Extrair ALÍQUOTA:
    valor_aliquota_match = re.search(r'ALÍQUOTA:\s+(.+)', text)
    if valor_aliquota_match:
        valor_aliquota_str = valor_aliquota_match.group(1)
        valor_aliquota_sem_formato = re.search(r'([\d.,]+)%', valor_aliquota_str)  # Ajuste aqui
        if valor_aliquota_sem_formato:
            valor_aliquota_sem_formato = valor_aliquota_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['aliquota'] = float(valor_aliquota_sem_formato)
        else:
            nf_data_valores['aliquota'] = 0.0  # Valor não encontrado ou não está no formato esperado


    # Extrair VALOR ISS:
    valor_iss_match = re.search(r'VALOR ISS:\s+(.+)', text)
    if valor_iss_match:
        valor_iss_str = valor_iss_match.group(1)
        valor_iss_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_iss_str)
        if valor_iss_sem_formato:
            valor_iss_sem_formato = valor_iss_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['valor_iss'] = float(valor_iss_sem_formato)
        else:
            nf_data_valores['valor_iss'] = 0.0  # Valor não encontrado ou não está no formato esperado 

    # Extrair VALOR ISS RETIDO:
    valor_iss_retido_match = re.search(r'RETIDO:\s+(.+)', text)
    if valor_iss_match:
        valor_iss_retido_str = valor_iss_retido_match.group(1)
        valor_iss_retido_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_iss_retido_str)
        if valor_iss_retido_sem_formato:
            valor_iss_retido_sem_formato = valor_iss_retido_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['valor_iss_retido'] = float(valor_iss_retido_sem_formato)
        else:
            nf_data_valores['valor_iss_retido'] = 0.0  # Valor não encontrado ou não está no formato esperado 

    # Extrair VALOR DESC. COND:
    valor_desc_cond_match = re.search(r'DESC. COND:\s+(.+)', text)
    if valor_desc_cond_match:
        valor_desc_cond_str = valor_desc_cond_match.group(1)
        valor_desc_cond_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_desc_cond_str)
        if valor_desc_cond_sem_formato:
            valor_desc_cond_sem_formato = valor_desc_cond_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['desc_cond'] = float(valor_desc_cond_sem_formato)
        else:
            nf_data_valores['desc_cond'] = 0.0  # Valor não encontrado ou não está no formato esperado
    
    # Extrair VALOR PIS:
    valor_pis_match = re.search(r'VALOR PIS:\s+(.+)', text)
    if valor_pis_match:
        valor_pis_str = valor_pis_match.group(1)
        valor_pis_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_pis_str)
        if valor_pis_sem_formato:
            valor_pis_sem_formato = valor_pis_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['valor_pis'] = float(valor_pis_sem_formato)
        else:
            nf_data_valores['valor_pis'] = 0.0  # Valor não encontrado ou não está no formato esperado
    
    # Extrair VALOR COFINS:
    valor_cofins_match = re.search(r'VALOR COFINS:\s+(.+)', text)
    if valor_cofins_match:
        valor_cofins_str = valor_cofins_match.group(1)
        valor_cofins_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_cofins_str)
        if valor_cofins_sem_formato:
            valor_cofins_sem_formato = valor_cofins_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['valor_cofins'] = float(valor_cofins_sem_formato)
        else:
            nf_data_valores['valor_cofins'] = 0.0  # Valor não encontrado ou não está no formato esperado
            
    # Extrair VALOR IR:
    valor_ir_match = re.search(r'VALOR IR:\s+(.+)', text)
    if valor_ir_match:
        valor_ir_str = valor_ir_match.group(1)
        valor_ir_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_ir_str)
        if valor_ir_sem_formato:
            valor_ir_sem_formato = valor_ir_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['valor_ir'] = float(valor_ir_sem_formato)
        else:
            nf_data_valores['valor_ir'] = 0.0  # Valor não encontrado ou não está no formato esperado
            
    # Extrair VALOR INSS:
    valor_inss_match = re.search(r'VALOR INSS:\s+(.+)', text)
    if valor_inss_match:
        valor_inss_str = valor_inss_match.group(1)
        valor_inss_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_inss_str)
        if valor_inss_sem_formato:
            valor_inss_sem_formato = valor_inss_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['valor_inss'] = float(valor_inss_sem_formato)
        else:
            nf_data_valores['valor_inss'] = 0.0  # Valor não encontrado ou não está no formato esperado
            
    # Extrair VALOR CSLL:
    valor_csll_match = re.search(r'VALOR CSLL:\s+(.+)', text)
    if valor_csll_match:
        valor_csll_str = valor_csll_match.group(1)
        valor_csll_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_csll_str)
        if valor_csll_sem_formato:
            valor_csll_sem_formato = valor_csll_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['valor_csll'] = float(valor_csll_sem_formato)
        else:
            nf_data_valores['valor_csll'] = 0.0  # Valor não encontrado ou não está no formato esperado
    
    # Extrair OUTRAS RETENÇÕES:
    outras_retencoes_match = re.search(r'OUTRAS RETENÇÕES:\s+(.+)', text)
    if outras_retencoes_match:
        outras_retencoes_str = outras_retencoes_match.group(1)
        outras_retencoes_sem_formato = re.search(r'R\$\s*([\d.,]+)', outras_retencoes_str)
        if outras_retencoes_sem_formato:
            outras_retencoes_sem_formato = outras_retencoes_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['outras_retencoes'] = float(outras_retencoes_sem_formato)
        else:
            nf_data_valores['outras_retencoes'] = 0.0  # Valor não encontrado ou não está no formato esperado
    
    
    # Extrair VALOR LÍQUIDO:
    valor_liquido_match = re.search(r'VALOR LÍQUIDO:\s+(.+)', text)
    if valor_liquido_match:
        valor_liquido_str = valor_liquido_match.group(1)
        valor_liquido_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_liquido_str)
        if valor_liquido_sem_formato:
            valor_liquido_sem_formato = valor_liquido_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['valor_liquido'] = float(valor_liquido_sem_formato)
        else:
            nf_data_valores['valor_liquido'] = 0.0  # Valor não encontrado ou não está no formato esperado
        

    return nf_data_valores


# 9. OUTRAS INFORMAÇOES / CRITICAS
def extract_fields_outras_info(text):
    nf_data_outras_informacoes = {}
    nf_data_outras_informacoes['secao'] = "9. OUTRAS INFORMAÇOES / CRITICAS"
    
    # Extrair EXIGIBILIDADE ISS:
    exigibilidade_iss_match = re.search(r'EXIGIBILIDADE ISS\s+(.+)', text)
    if exigibilidade_iss_match:
        exigibilidade_iss_value = exigibilidade_iss_match.group(1).strip()
        nf_data_outras_informacoes['exigibilidade_iss'] = exigibilidade_iss_value
        
    # Extrair REGIME TRIBUTAÇÃO:
    regime_tributacao_match = re.search(r'REGIME TRIBUTAÇÃO\s+(.+)', text)
    if regime_tributacao_match:
        regime_tributacao_value = regime_tributacao_match.group(1).strip()
        nf_data_outras_informacoes['regime_tributacao'] = regime_tributacao_value
    
    # Extrair SIMPLES NACIONAL:
    simples_nacional_match = re.search(r'SIMPLES NACIONAL\s+(.+)', text)
    if simples_nacional_match:
        simples_nacional_value = simples_nacional_match.group(1).strip()
        nf_data_outras_informacoes['simples_nacional'] = simples_nacional_value
        
        
    # Extrair ISSQN RETIDO:
    local_prestacao_servico_match = re.search(r'ISSQN RETIDO\s+(.+)', text)
    if local_prestacao_servico_match:
        local_prestacao_servico_value = local_prestacao_servico_match.group(1).strip()
        nf_data_outras_informacoes['issqn_retido'] = local_prestacao_servico_value        
        
    
    # Extrair LOCAL PRESTAÇÃO SERVIÇO:
    local_prestacao_servico_match = re.search(r'LOCAL\. PRESTAÇÃO\s+SERVIÇO\s+(.+)', text)
    if local_prestacao_servico_match:
        local_prestacao_servico_value = local_prestacao_servico_match.group(1).strip()
        nf_data_outras_informacoes['local_prestacao_servico'] = local_prestacao_servico_value
    
    
    
    # Extrair LOCAL INCIDÊNCIA:
    local_incidencia_match = re.search(r'LOCAL INCIDÊNCIA\s+(.+)', text)
    if local_incidencia_match:
        local_incidencia_value = local_incidencia_match.group(1).strip()
        nf_data_outras_informacoes['local_incidencia'] = local_incidencia_value
   
    
    return nf_data_outras_informacoes

In [ ]:
        # secao: 5 - VALOR TOTAL
        data_valor_total = {}
        #data_valor_total = processa_total()
        father_value = "4_frame_valor_total"
        section = "5. VALOR TOTAL"
        
        result = extract_fields_box(modelo, father_value, section)
        if result:
            data_valor_total.update(result)
        
        # secao: 6 - CNAE e Item da Lista de Serviços
        data_CNAE = {}
        data_CNAE = processa_cnae_itens()

        # secao: 7 - VALORES E IMPOSTOS & 9 - OUTRAS INFORMAÇOES / CRITICAS
        data_valores = {}
        father_value = "5_frame_valores_impostos"
        section = "7. VALORES E IMPOSTOS"

        result = extract_fields_box(modelo, father_value, section)
        if result:
            data_valores.update(result)
            
        # secao: 8 - DADOS COMPLEMENTARES"
        data_dados_complementares = {}
        f_father = "5_frame_dados_complementares"
        section = "8. DADOS COMPLEMENTARES"

        data_dados_complementares = extract_dados_comple_obs(modelo, f_father, section)                                           
                                
                                
        # secao: 9 - OUTRAS INFORMAÇOES / CRITICAS
        data_outras_informacoes = {}
        father_value = "5_frame_inf_criticas"
        section = "9. OUTRAS INFORMAÇOES / CRITICAS"

        result = extract_fields_box(modelo, father_value, section)
        if result:
            data_outras_informacoes.update(result)                        
                            

        # secao: 10. OBSERVACOES
        data_observacao = {}
        f_father = "5_frame_observacao"
        section = "10. OBSERVACOES"

        data_observacao = extract_dados_comple_obs(modelo, f_father, section)

## Outras funcoes deprecateds

In [ ]:
def find_value_after_keyword_out_frame(keyword, text_list, default_keyword_list=None):
    try:
        index = text_list.index(keyword)
        # Verifica se o índice seguinte está dentro da lista
        if index + 1 < len(text_list):
            # Verifica se o valor seguinte não é outra keyword da lista default_keyword_list
            if text_list[index + 1] not in default_keyword_list:
                return text_list[index + 1]
            else:
                return "Valor não encontrado"
        else:
            return "Keyword é o último elemento, valor não encontrado"
    except ValueError:
        if default_keyword_list:
            try:
                index = text_list.index(default_keyword_list[-1])
                return text_list[index - 1]
            except ValueError:
                return "Keyword não encontrada"
        else:
            return "Keyword não encontrada"


In [ ]:
def find_value_after_keyword(keyword, text_list, default_keyword_list=None):
    try:
        index = text_list.index(keyword)
        # Verifica se o índice seguinte está dentro da lista
        if index + 1 < len(text_list):
            # Verifica se o valor seguinte não é outra keyword da lista default_keyword_list
            if text_list[index + 1] not in default_keyword_list:
                return text_list[index + 1]
            else:
                return "Valor não encontrado"
        else:
            return "Keyword é o último elemento, valor não encontrado"
    except ValueError:
        if default_keyword_list:
            try:
                index = text_list.index(default_keyword_list[-1])
                return text_list[index - 1]
            except ValueError:
                return "Keyword não encontrada"
        else:
            return "Keyword não encontrada"